### Imports and Set Flag

In [273]:
import os
import pandas as pd
import polars as pl
import numpy as np

from tqdm import tqdm


# Add training directory to path for imports
PROJ_DIR = os.path.dirname(os.getcwd())
TRAINING_DIR = os.path.join(PROJ_DIR, "training")
sys.path.append(TRAINING_DIR)

from config import config
from lightgbm import LGBMRegressor


### Environment Setup ###
IS_KAGGLE = False  # Flag to switch between environments


# Progress bar - only create if running locally
N_TEST_DAYS = 20   # Number of days in test set
progress_bar = None if IS_KAGGLE else tqdm(total=N_TEST_DAYS, desc='Processing days')

Processing days:   0%|          | 0/21 [00:00<?, ?it/s]

### Environment Dependent Path Setup

In [274]:
# Setup paths based on environment
if IS_KAGGLE:
    data_path = '/kaggle/input/jane-street-real-time-market-data-forecasting'
    trained_models_path = "/kaggle/input/janestreet-models/trained_models/"
else:
    # For local testing
    import sys

    # Path to main project directory
    PROJ_DIR = os.path.dirname(os.getcwd())
    
    # Add paths to system path
    sys.path.append(os.path.join(PROJ_DIR, "jane-street-real-time-market-data-forecasting"))
    sys.path.append(os.path.join(PROJ_DIR, "training", "src", "utils"))
    
    # Set data path to our local test data
    data_path = os.path.join(os.getcwd(), "local_test_data")
    
    # Import local testing metrics
    from metrics import r2_score_weighted

import kaggle_evaluation.jane_street_inference_server

### Pre-processing

#### A global dictionary that initialises with the last seen values for the set we are about to see.

In [275]:
last_known_values = {
    (0, 'feature_00'): 3.900035,
    (0, 'feature_01'): -0.570511,
    (0, 'feature_02'): 3.300193,
    (0, 'feature_03'): 3.526469,
    (0, 'feature_04'): -1.407291,
    (0, 'feature_05'): 1.516465,
    (0, 'feature_06'): 1.263282,
    (0, 'feature_07'): 0.527582,
    (0, 'feature_08'): -1.472504,
    (0, 'feature_09'): 11.000000,
    (0, 'feature_10'): 7.000000,
    (0, 'feature_11'): 76.000000,
    (0, 'feature_12'): 0.024299,
    (0, 'feature_13'): -0.042058,
    (0, 'feature_14'): 0.079692,
    (0, 'feature_15'): -0.897486,
    (0, 'feature_16'): -0.731225,
    (0, 'feature_17'): -0.539531,
    (0, 'feature_18'): -0.412632,
    (0, 'feature_19'): 2.683558,
    (0, 'feature_20'): 0.747947,
    (0, 'feature_21'): -0.123195,
    (0, 'feature_22'): 1.410305,
    (0, 'feature_23'): 0.544582,
    (0, 'feature_24'): 0.760277,
    (0, 'feature_25'): 0.463058,
    (0, 'feature_26'): 1.264379,
    (0, 'feature_27'): 1.245374,
    (0, 'feature_28'): 0.494950,
    (0, 'feature_29'): -0.719145,
    (0, 'feature_30'): -0.935777,
    (0, 'feature_31'): -0.183156,
    (0, 'feature_32'): 2.838497,
    (0, 'feature_33'): -0.355857,
    (0, 'feature_34'): 3.543872,
    (0, 'feature_35'): 3.738325,
    (0, 'feature_36'): -1.264353,
    (0, 'feature_37'): -1.751738,
    (0, 'feature_38'): -1.182681,
    (0, 'feature_39'): 1.489907,
    (0, 'feature_40'): 0.366242,
    (0, 'feature_41'): 0.975179,
    (0, 'feature_42'): 0.019992,
    (0, 'feature_43'): 0.130387,
    (0, 'feature_44'): -0.572785,
    (0, 'feature_45'): 1.303157,
    (0, 'feature_46'): 1.216478,
    (0, 'feature_47'): 1.020091,
    (0, 'feature_48'): 0.397092,
    (0, 'feature_49'): 0.989084,
    (0, 'feature_50'): 2.279053,
    (0, 'feature_51'): 0.906493,
    (0, 'feature_52'): 1.381002,
    (0, 'feature_53'): -0.429956,
    (0, 'feature_54'): 0.965162,
    (0, 'feature_55'): 0.064794,
    (0, 'feature_56'): 1.056885,
    (0, 'feature_57'): 0.829070,
    (0, 'feature_58'): 2.492225,
    (0, 'feature_59'): 0.368479,
    (0, 'feature_60'): 1.236967,
    (0, 'feature_61'): -1.275181,
    (0, 'feature_62'): -0.483084,
    (0, 'feature_63'): -0.372049,
    (0, 'feature_64'): -0.410466,
    (0, 'feature_65'): 0.043027,
    (0, 'feature_66'): 3.793989,
    (0, 'feature_67'): -0.145327,
    (0, 'feature_68'): -0.162733,
    (0, 'feature_69'): -0.042974,
    (0, 'feature_70'): 0.173764,
    (0, 'feature_71'): 0.108309,
    (0, 'feature_72'): 0.233715,
    (0, 'feature_73'): -0.181749,
    (0, 'feature_74'): -0.036662,
    (0, 'feature_75'): 0.185636,
    (0, 'feature_76'): 0.339268,
    (0, 'feature_77'): 0.000957,
    (0, 'feature_78'): 0.149368,
    (1, 'feature_00'): 3.811610,
    (1, 'feature_01'): -1.074320,
    (1, 'feature_02'): 3.478107,
    (1, 'feature_03'): 3.381300,
    (1, 'feature_04'): -1.039974,
    (1, 'feature_05'): 1.012028,
    (1, 'feature_06'): 0.813179,
    (1, 'feature_07'): 0.469078,
    (1, 'feature_08'): -1.505578,
    (1, 'feature_09'): 11.000000,
    (1, 'feature_10'): 7.000000,
    (1, 'feature_11'): 76.000000,
    (1, 'feature_12'): 0.062384,
    (1, 'feature_13'): 0.221948,
    (1, 'feature_14'): 0.411186,
    (1, 'feature_15'): -0.782406,
    (1, 'feature_16'): -0.613157,
    (1, 'feature_17'): -0.718652,
    (1, 'feature_18'): -0.356973,
    (1, 'feature_19'): 4.364563,
    (1, 'feature_20'): 1.089405,
    (1, 'feature_21'): -0.044259,
    (1, 'feature_22'): 0.895434,
    (1, 'feature_23'): 0.562779,
    (1, 'feature_24'): 0.826629,
    (1, 'feature_25'): 0.628654,
    (1, 'feature_26'): -1.690828,
    (1, 'feature_27'): -0.260739,
    (1, 'feature_28'): 0.938939,
    (1, 'feature_29'): -0.603682,
    (1, 'feature_30'): -0.602102,
    (1, 'feature_31'): -0.077681,
    (1, 'feature_32'): 1.826113,
    (1, 'feature_33'): -1.043039,
    (1, 'feature_34'): 3.028466,
    (1, 'feature_35'): 2.796797,
    (1, 'feature_36'): -1.236207,
    (1, 'feature_37'): -1.713611,
    (1, 'feature_38'): -1.576142,
    (1, 'feature_39'): 1.645427,
    (1, 'feature_40'): -0.035206,
    (1, 'feature_41'): 1.021414,
    (1, 'feature_42'): 0.858585,
    (1, 'feature_43'): 0.049966,
    (1, 'feature_44'): 0.316280,
    (1, 'feature_45'): 1.466974,
    (1, 'feature_46'): 1.228570,
    (1, 'feature_47'): 0.852935,
    (1, 'feature_48'): 0.446486,
    (1, 'feature_49'): 0.738544,
    (1, 'feature_50'): 1.573048,
    (1, 'feature_51'): -0.265453,
    (1, 'feature_52'): 1.283118,
    (1, 'feature_53'): 0.876510,
    (1, 'feature_54'): -0.702592,
    (1, 'feature_55'): 0.315744,
    (1, 'feature_56'): 1.890897,
    (1, 'feature_57'): 1.753430,
    (1, 'feature_58'): 1.714161,
    (1, 'feature_59'): 0.389807,
    (1, 'feature_60'): 0.733676,
    (1, 'feature_61'): -1.275181,
    (1, 'feature_62'): -0.617651,
    (1, 'feature_63'): -0.354253,
    (1, 'feature_64'): -0.334916,
    (1, 'feature_65'): -0.503207,
    (1, 'feature_66'): 3.177802,
    (1, 'feature_67'): -0.080306,
    (1, 'feature_68'): -0.051042,
    (1, 'feature_69'): 0.021711,
    (1, 'feature_70'): 0.308090,
    (1, 'feature_71'): 0.376416,
    (1, 'feature_72'): 0.576502,
    (1, 'feature_73'): -0.213916,
    (1, 'feature_74'): -0.035290,
    (1, 'feature_75'): 0.268288,
    (1, 'feature_76'): 0.420007,
    (1, 'feature_77'): -0.151749,
    (1, 'feature_78'): 0.022894,
    (2, 'feature_00'): 3.040113,
    (2, 'feature_01'): -0.897422,
    (2, 'feature_02'): 3.331955,
    (2, 'feature_03'): 3.369095,
    (2, 'feature_04'): -0.884140,
    (2, 'feature_05'): 1.322768,
    (2, 'feature_06'): 1.101791,
    (2, 'feature_07'): 0.557570,
    (2, 'feature_08'): -1.650404,
    (2, 'feature_09'): 81.000000,
    (2, 'feature_10'): 2.000000,
    (2, 'feature_11'): 59.000000,
    (2, 'feature_12'): 0.184938,
    (2, 'feature_13'): 0.074110,
    (2, 'feature_14'): 0.391336,
    (2, 'feature_15'): -0.722296,
    (2, 'feature_16'): -0.594649,
    (2, 'feature_17'): -0.642553,
    (2, 'feature_18'): -1.037938,
    (2, 'feature_19'): 2.631149,
    (2, 'feature_20'): 0.358467,
    (2, 'feature_21'): -0.149106,
    (2, 'feature_22'): 0.403424,
    (2, 'feature_23'): 1.090115,
    (2, 'feature_24'): 0.847028,
    (2, 'feature_25'): 0.393974,
    (2, 'feature_26'): 0.735956,
    (2, 'feature_27'): 0.315081,
    (2, 'feature_28'): -0.149950,
    (2, 'feature_29'): -0.557452,
    (2, 'feature_30'): -0.516999,
    (2, 'feature_31'): -0.220321,
    (2, 'feature_32'): 2.796667,
    (2, 'feature_33'): -0.062794,
    (2, 'feature_34'): 2.568594,
    (2, 'feature_35'): 2.993846,
    (2, 'feature_36'): -0.551795,
    (2, 'feature_37'): -0.535115,
    (2, 'feature_38'): -0.745381,
    (2, 'feature_39'): 0.815272,
    (2, 'feature_40'): -0.127548,
    (2, 'feature_41'): 1.235851,
    (2, 'feature_42'): -0.637731,
    (2, 'feature_43'): -0.846532,
    (2, 'feature_44'): -0.625438,
    (2, 'feature_45'): 0.507483,
    (2, 'feature_46'): 0.004066,
    (2, 'feature_47'): -0.233026,
    (2, 'feature_48'): -0.163131,
    (2, 'feature_49'): 0.114962,
    (2, 'feature_50'): 0.813470,
    (2, 'feature_51'): -1.250553,
    (2, 'feature_52'): 1.886529,
    (2, 'feature_53'): -0.771083,
    (2, 'feature_54'): 0.050516,
    (2, 'feature_55'): -0.802760,
    (2, 'feature_56'): 0.576811,
    (2, 'feature_57'): 0.450892,
    (2, 'feature_58'): -0.627272,
    (2, 'feature_59'): -0.411310,
    (2, 'feature_60'): 0.094082,
    (2, 'feature_61'): -1.275181,
    (2, 'feature_62'): -0.422706,
    (2, 'feature_63'): -0.258431,
    (2, 'feature_64'): -0.411771,
    (2, 'feature_65'): -0.844565,
    (2, 'feature_66'): 2.467115,
    (2, 'feature_67'): 0.143665,
    (2, 'feature_68'): 0.066121,
    (2, 'feature_69'): 0.423912,
    (2, 'feature_70'): 0.194452,
    (2, 'feature_71'): 0.041308,
    (2, 'feature_72'): 0.505649,
    (2, 'feature_73'): -0.209658,
    (2, 'feature_74'): -0.127630,
    (2, 'feature_75'): 1.101242,
    (2, 'feature_76'): 0.781673,
    (2, 'feature_77'): 0.162630,
    (2, 'feature_78'): 0.185932,
    (3, 'feature_00'): 3.513022,
    (3, 'feature_01'): -0.946374,
    (3, 'feature_02'): 2.987281,
    (3, 'feature_03'): 3.439636,
    (3, 'feature_04'): -1.032906,
    (3, 'feature_05'): 1.220087,
    (3, 'feature_06'): 1.150397,
    (3, 'feature_07'): 0.647740,
    (3, 'feature_08'): -1.477099,
    (3, 'feature_09'): 4.000000,
    (3, 'feature_10'): 3.000000,
    (3, 'feature_11'): 11.000000,
    (3, 'feature_12'): 0.364421,
    (3, 'feature_13'): 1.244820,
    (3, 'feature_14'): 0.791655,
    (3, 'feature_15'): -0.693602,
    (3, 'feature_16'): -0.558490,
    (3, 'feature_17'): -0.693386,
    (3, 'feature_18'): -2.108607,
    (3, 'feature_19'): 3.330006,
    (3, 'feature_20'): 0.024631,
    (3, 'feature_21'): -0.113637,
    (3, 'feature_22'): 0.241995,
    (3, 'feature_23'): 0.589095,
    (3, 'feature_24'): 0.598057,
    (3, 'feature_25'): 0.083294,
    (3, 'feature_26'): -0.015525,
    (3, 'feature_27'): -0.332640,
    (3, 'feature_28'): -0.639177,
    (3, 'feature_29'): -0.556940,
    (3, 'feature_30'): -0.741762,
    (3, 'feature_31'): -0.100452,
    (3, 'feature_32'): 2.979039,
    (3, 'feature_33'): -0.824419,
    (3, 'feature_34'): 2.709861,
    (3, 'feature_35'): 3.000818,
    (3, 'feature_36'): -1.327255,
    (3, 'feature_37'): -1.807550,
    (3, 'feature_38'): -1.123411,
    (3, 'feature_39'): 0.442940,
    (3, 'feature_40'): -0.729237,
    (3, 'feature_41'): 1.111405,
    (3, 'feature_42'): 0.319185,
    (3, 'feature_43'): -0.266130,
    (3, 'feature_44'): -0.176116,
    (3, 'feature_45'): 0.966503,
    (3, 'feature_46'): 1.458072,
    (3, 'feature_47'): 0.456921,
    (3, 'feature_48'): 0.120908,
    (3, 'feature_49'): 0.660205,
    (3, 'feature_50'): 0.359981,
    (3, 'feature_51'): 0.918100,
    (3, 'feature_52'): 0.964227,
    (3, 'feature_53'): 0.283272,
    (3, 'feature_54'): 0.155154,
    (3, 'feature_55'): -0.036079,
    (3, 'feature_56'): 1.543043,
    (3, 'feature_57'): 0.918653,
    (3, 'feature_58'): 0.990925,
    (3, 'feature_59'): 0.527624,
    (3, 'feature_60'): 0.798666,
    (3, 'feature_61'): -1.275181,
    (3, 'feature_62'): -0.387687,
    (3, 'feature_63'): -0.396789,
    (3, 'feature_64'): -0.411218,
    (3, 'feature_65'): -2.061536,
    (3, 'feature_66'): 2.469001,
    (3, 'feature_67'): 0.042349,
    (3, 'feature_68'): 0.918992,
    (3, 'feature_69'): 0.170002,
    (3, 'feature_70'): 0.801904,
    (3, 'feature_71'): 1.245985,
    (3, 'feature_72'): 1.122003,
    (3, 'feature_73'): -0.003449,
    (3, 'feature_74'): 0.125776,
    (3, 'feature_75'): 0.991120,
    (3, 'feature_76'): 1.070261,
    (3, 'feature_77'): 0.300665,
    (3, 'feature_78'): 0.420546,
    (4, 'feature_00'): 3.367398,
    (4, 'feature_01'): -0.543882,
    (4, 'feature_02'): 2.933169,
    (4, 'feature_03'): 3.240888,
    (4, 'feature_04'): -1.274680,
    (4, 'feature_05'): 0.637455,
    (4, 'feature_06'): 0.475879,
    (4, 'feature_07'): 0.138429,
    (4, 'feature_08'): -1.008747,
    (4, 'feature_09'): 15.000000,
    (4, 'feature_10'): 1.000000,
    (4, 'feature_11'): 9.000000,
    (4, 'feature_12'): 54.756905,
    (4, 'feature_13'): 69.933838,
    (4, 'feature_14'): 40.014042,
    (4, 'feature_15'): -0.973268,
    (4, 'feature_16'): -0.719266,
    (4, 'feature_17'): -0.808689,
    (4, 'feature_18'): -0.704479,
    (4, 'feature_19'): 0.593011,
    (4, 'feature_20'): -1.811700,
    (4, 'feature_21'): -0.238110,
    (4, 'feature_22'): -0.962137,
    (4, 'feature_23'): 1.023004,
    (4, 'feature_24'): 7.449043,
    (4, 'feature_25'): 3.114555,
    (4, 'feature_26'): -1.401530,
    (4, 'feature_27'): -1.771829,
    (4, 'feature_28'): -0.675125,
    (4, 'feature_29'): -0.406465,
    (4, 'feature_30'): -0.962647,
    (4, 'feature_31'): -0.231196,
    (4, 'feature_32'): 2.443467,
    (4, 'feature_33'): -1.775033,
    (4, 'feature_34'): 2.511347,
    (4, 'feature_35'): 2.961508,
    (4, 'feature_36'): -2.160760,
    (4, 'feature_37'): -1.291191,
    (4, 'feature_38'): -0.445911,
    (4, 'feature_39'): 1.568947,
    (4, 'feature_40'): 1.424612,
    (4, 'feature_41'): 1.535967,
    (4, 'feature_42'): 0.158198,
    (4, 'feature_43'): -0.204978,
    (4, 'feature_44'): 0.684730,
    (4, 'feature_45'): 1.109982,
    (4, 'feature_46'): -0.346890,
    (4, 'feature_47'): 0.639065,
    (4, 'feature_48'): 1.290502,
    (4, 'feature_49'): 1.319663,
    (4, 'feature_50'): 1.640022,
    (4, 'feature_51'): 1.016770,
    (4, 'feature_52'): 1.816369,
    (4, 'feature_53'): 0.590088,
    (4, 'feature_54'): 0.091355,
    (4, 'feature_55'): 0.722525,
    (4, 'feature_56'): 1.671262,
    (4, 'feature_57'): -0.131687,
    (4, 'feature_58'): 1.216806,
    (4, 'feature_59'): 1.366238,
    (4, 'feature_60'): 1.886808,
    (4, 'feature_61'): -1.275181,
    (4, 'feature_62'): -0.316556,
    (4, 'feature_63'): -0.306589,
    (4, 'feature_64'): -0.497232,
    (4, 'feature_65'): -0.690375,
    (4, 'feature_66'): 0.456040,
    (4, 'feature_67'): 48.250179,
    (4, 'feature_68'): 27.921423,
    (4, 'feature_69'): 34.542526,
    (4, 'feature_70'): 36.213871,
    (4, 'feature_71'): 107.820000,
    (4, 'feature_72'): 56.564766,
    (4, 'feature_73'): -2.247483,
    (4, 'feature_74'): -1.748126,
    (4, 'feature_75'): -0.674504,
    (4, 'feature_76'): -0.614881,
    (4, 'feature_77'): -1.096028,
    (4, 'feature_78'): -1.827890,
    (5, 'feature_00'): 3.599698,
    (5, 'feature_01'): -0.198784,
    (5, 'feature_02'): 3.139261,
    (5, 'feature_03'): 3.428588,
    (5, 'feature_04'): -1.240746,
    (5, 'feature_05'): 1.028296,
    (5, 'feature_06'): 0.761402,
    (5, 'feature_07'): 0.428136,
    (5, 'feature_08'): -1.383428,
    (5, 'feature_09'): 2.000000,
    (5, 'feature_10'): 10.000000,
    (5, 'feature_11'): 171.000000,
    (5, 'feature_12'): 0.614748,
    (5, 'feature_13'): 0.581167,
    (5, 'feature_14'): 0.384525,
    (5, 'feature_15'): -0.797240,
    (5, 'feature_16'): -0.459516,
    (5, 'feature_17'): -0.677272,
    (5, 'feature_18'): -0.801000,
    (5, 'feature_19'): 2.505128,
    (5, 'feature_20'): 0.426395,
    (5, 'feature_21'): -0.089614,
    (5, 'feature_22'): 0.707120,
    (5, 'feature_23'): 0.256785,
    (5, 'feature_24'): 1.004956,
    (5, 'feature_25'): 1.044359,
    (5, 'feature_26'): -0.836900,
    (5, 'feature_27'): -0.004220,
    (5, 'feature_28'): 0.771441,
    (5, 'feature_29'): -0.635638,
    (5, 'feature_30'): -1.176022,
    (5, 'feature_31'): -0.124830,
    (5, 'feature_32'): 2.743850,
    (5, 'feature_33'): -0.629490,
    (5, 'feature_34'): 3.394122,
    (5, 'feature_35'): 3.359625,
    (5, 'feature_36'): -0.078565,
    (5, 'feature_37'): -0.356584,
    (5, 'feature_38'): -0.478561,
    (5, 'feature_39'): 1.070155,
    (5, 'feature_40'): 0.301086,
    (5, 'feature_41'): 0.453633,
    (5, 'feature_42'): -0.364542,
    (5, 'feature_43'): -0.695173,
    (5, 'feature_44'): -0.658889,
    (5, 'feature_45'): 0.286304,
    (5, 'feature_46'): 0.135488,
    (5, 'feature_47'): -0.105252,
    (5, 'feature_48'): -0.728828,
    (5, 'feature_49'): 0.018415,
    (5, 'feature_50'): 1.153624,
    (5, 'feature_51'): -0.250689,
    (5, 'feature_52'): -1.476667,
    (5, 'feature_53'): -0.759694,
    (5, 'feature_54'): -1.223646,
    (5, 'feature_55'): -1.003595,
    (5, 'feature_56'): 0.504926,
    (5, 'feature_57'): -0.260308,
    (5, 'feature_58'): -0.231775,
    (5, 'feature_59'): -1.246848,
    (5, 'feature_60'): -0.171216,
    (5, 'feature_61'): -1.275181,
    (5, 'feature_62'): -0.336089,
    (5, 'feature_63'): -0.303831,
    (5, 'feature_64'): -0.405200,
    (5, 'feature_65'): -0.585282,
    (5, 'feature_66'): 2.330231,
    (5, 'feature_67'): 0.838703,
    (5, 'feature_68'): 0.948516,
    (5, 'feature_69'): 0.393164,
    (5, 'feature_70'): 0.694935,
    (5, 'feature_71'): 0.024679,
    (5, 'feature_72'): 0.414600,
    (5, 'feature_73'): 0.148232,
    (5, 'feature_74'): 0.260746,
    (5, 'feature_75'): 1.769529,
    (5, 'feature_76'): 1.787355,
    (5, 'feature_77'): 0.797333,
    (5, 'feature_78'): 0.697259,
    (6, 'feature_00'): 3.559319,
    (6, 'feature_01'): -0.814562,
    (6, 'feature_02'): 2.848542,
    (6, 'feature_03'): 2.988220,
    (6, 'feature_04'): -0.729600,
    (6, 'feature_05'): 1.255914,
    (6, 'feature_06'): 0.499571,
    (6, 'feature_07'): 0.546117,
    (6, 'feature_08'): -0.996653,
    (6, 'feature_09'): 12.000000,
    (6, 'feature_10'): 4.000000,
    (6, 'feature_11'): 34.000000,
    (6, 'feature_12'): -0.441824,
    (6, 'feature_13'): -0.182938,
    (6, 'feature_14'): -0.187711,
    (6, 'feature_15'): -0.839570,
    (6, 'feature_16'): -0.257413,
    (6, 'feature_17'): -0.499263,
    (6, 'feature_18'): -2.007128,
    (6, 'feature_19'): 2.597237,
    (6, 'feature_20'): -1.788885,
    (6, 'feature_21'): -0.219607,
    (6, 'feature_22'): -0.222722,
    (6, 'feature_23'): -0.254798,
    (6, 'feature_24'): 1.357037,
    (6, 'feature_25'): 0.761813,
    (6, 'feature_26'): -0.449724,
    (6, 'feature_27'): -0.515842,
    (6, 'feature_28'): -0.734519,
    (6, 'feature_29'): -0.649157,
    (6, 'feature_30'): -0.650469,
    (6, 'feature_31'): -0.289036,
    (6, 'feature_32'): 1.924483,
    (6, 'feature_33'): -0.160662,
    (6, 'feature_34'): 2.058397,
    (6, 'feature_35'): 1.986243,
    (6, 'feature_36'): 1.062441,
    (6, 'feature_37'): -1.131619,
    (6, 'feature_38'): -1.796284,
    (6, 'feature_39'): -0.094636,
    (6, 'feature_40'): 0.686704,
    (6, 'feature_41'): 1.079657,
    (6, 'feature_42'): -0.140163,
    (6, 'feature_43'): -0.512216,
    (6, 'feature_44'): -0.199946,
    (6, 'feature_45'): 0.870460,
    (6, 'feature_46'): 0.395649,
    (6, 'feature_47'): 0.233553,
    (6, 'feature_48'): 0.348791,
    (6, 'feature_49'): 0.112091,
    (6, 'feature_50'): 0.497057,
    (6, 'feature_51'): 0.236209,
    (6, 'feature_52'): 1.649183,
    (6, 'feature_53'): -0.523115,
    (6, 'feature_54'): 0.090676,
    (6, 'feature_55'): -1.058576,
    (6, 'feature_56'): 1.544855,
    (6, 'feature_57'): 0.094512,
    (6, 'feature_58'): 0.363805,
    (6, 'feature_59'): 0.174838,
    (6, 'feature_60'): 0.007120,
    (6, 'feature_61'): -1.275181,
    (6, 'feature_62'): -0.460570,
    (6, 'feature_63'): -0.182173,
    (6, 'feature_64'): -0.402997,
    (6, 'feature_65'): -1.921988,
    (6, 'feature_66'): 2.318596,
    (6, 'feature_67'): -0.479528,
    (6, 'feature_68'): -0.161519,
    (6, 'feature_69'): -0.218970,
    (6, 'feature_70'): -0.345637,
    (6, 'feature_71'): -0.100935,
    (6, 'feature_72'): -0.210007,
    (6, 'feature_73'): 0.145250,
    (6, 'feature_74'): 0.475562,
    (6, 'feature_75'): 4.070587,
    (6, 'feature_76'): 1.947849,
    (6, 'feature_77'): 1.754957,
    (6, 'feature_78'): 1.586910,
    (7, 'feature_00'): 3.479839,
    (7, 'feature_01'): -0.785468,
    (7, 'feature_02'): 3.363475,
    (7, 'feature_03'): 3.312040,
    (7, 'feature_04'): -1.222322,
    (7, 'feature_05'): 1.432833,
    (7, 'feature_06'): 1.242253,
    (7, 'feature_07'): 0.530978,
    (7, 'feature_08'): -1.829732,
    (7, 'feature_09'): 11.000000,
    (7, 'feature_10'): 7.000000,
    (7, 'feature_11'): 76.000000,
    (7, 'feature_12'): 0.495084,
    (7, 'feature_13'): 0.368698,
    (7, 'feature_14'): 0.982141,
    (7, 'feature_15'): -0.661350,
    (7, 'feature_16'): -0.361839,
    (7, 'feature_17'): -0.806280,
    (7, 'feature_18'): 0.269644,
    (7, 'feature_19'): 2.691852,
    (7, 'feature_20'): 0.358813,
    (7, 'feature_21'): -0.082840,
    (7, 'feature_22'): 1.416041,
    (7, 'feature_23'): 1.366390,
    (7, 'feature_24'): 0.421112,
    (7, 'feature_25'): -0.592694,
    (7, 'feature_26'): -0.154673,
    (7, 'feature_27'): 0.237162,
    (7, 'feature_28'): 0.295311,
    (7, 'feature_29'): -0.755904,
    (7, 'feature_30'): -0.542118,
    (7, 'feature_31'): -0.064503,
    (7, 'feature_32'): 2.531194,
    (7, 'feature_33'): -1.252443,
    (7, 'feature_34'): 3.368920,
    (7, 'feature_35'): 2.859969,
    (7, 'feature_36'): -0.848237,
    (7, 'feature_37'): -1.179709,
    (7, 'feature_38'): -1.136813,
    (7, 'feature_39'): 0.964657,
    (7, 'feature_40'): 0.355542,
    (7, 'feature_41'): 0.545381,
    (7, 'feature_42'): 0.191387,
    (7, 'feature_43'): -0.299540,
    (7, 'feature_44'): 0.602647,
    (7, 'feature_45'): 1.088200,
    (7, 'feature_46'): 0.789589,
    (7, 'feature_47'): 0.656871,
    (7, 'feature_48'): 0.629058,
    (7, 'feature_49'): 0.530044,
    (7, 'feature_50'): 0.394509,
    (7, 'feature_51'): 0.704461,
    (7, 'feature_52'): 1.307555,
    (7, 'feature_53'): 1.116410,
    (7, 'feature_54'): 0.443715,
    (7, 'feature_55'): 0.892270,
    (7, 'feature_56'): 0.624416,
    (7, 'feature_57'): 1.762137,
    (7, 'feature_58'): 1.735449,
    (7, 'feature_59'): 1.642416,
    (7, 'feature_60'): 0.977975,
    (7, 'feature_61'): -1.275181,
    (7, 'feature_62'): -0.461450,
    (7, 'feature_63'): -0.265461,
    (7, 'feature_64'): -0.275533,
    (7, 'feature_65'): 0.499440,
    (7, 'feature_66'): 3.033792,
    (7, 'feature_67'): 0.287140,
    (7, 'feature_68'): 0.377223,
    (7, 'feature_69'): 0.551442,
    (7, 'feature_70'): 0.594526,
    (7, 'feature_71'): 0.167041,
    (7, 'feature_72'): 0.997458,
    (7, 'feature_73'): -1.258975,
    (7, 'feature_74'): -1.309964,
    (7, 'feature_75'): -1.105459,
    (7, 'feature_76'): -1.121214,
    (7, 'feature_77'): -2.386290,
    (7, 'feature_78'): -1.660830,
    (8, 'feature_00'): 3.099889,
    (8, 'feature_01'): -0.510877,
    (8, 'feature_02'): 3.339279,
    (8, 'feature_03'): 3.415692,
    (8, 'feature_04'): -0.424228,
    (8, 'feature_05'): 0.968985,
    (8, 'feature_06'): 0.882114,
    (8, 'feature_07'): 0.473210,
    (8, 'feature_08'): -1.600333,
    (8, 'feature_09'): 15.000000,
    (8, 'feature_10'): 1.000000,
    (8, 'feature_11'): 62.000000,
    (8, 'feature_12'): 0.311423,
    (8, 'feature_13'): -0.146260,
    (8, 'feature_14'): -0.089695,
    (8, 'feature_15'): -0.927841,
    (8, 'feature_16'): -0.785175,
    (8, 'feature_17'): -1.037648,
    (8, 'feature_18'): -0.277144,
    (8, 'feature_19'): 2.067143,
    (8, 'feature_20'): 1.040143,
    (8, 'feature_21'): -0.024608,
    (8, 'feature_22'): 0.533892,
    (8, 'feature_23'): 0.137955,
    (8, 'feature_24'): 1.786227,
    (8, 'feature_25'): 1.220445,
    (8, 'feature_26'): 0.448663,
    (8, 'feature_27'): -0.712266,
    (8, 'feature_28'): -1.073628,
    (8, 'feature_29'): -0.475337,
    (8, 'feature_30'): -0.671129,
    (8, 'feature_31'): -0.020562,
    (8, 'feature_32'): 3.733887,
    (8, 'feature_33'): -0.170612,
    (8, 'feature_34'): 3.231449,
    (8, 'feature_35'): 3.148327,
    (8, 'feature_36'): -1.213138,
    (8, 'feature_37'): -0.839371,
    (8, 'feature_38'): -1.039740,
    (8, 'feature_39'): -0.244927,
    (8, 'feature_40'): -0.359087,
    (8, 'feature_41'): 0.204954,
    (8, 'feature_42'): -0.194007,
    (8, 'feature_43'): -0.196910,
    (8, 'feature_44'): -1.002442,
    (8, 'feature_45'): 1.752646,
    (8, 'feature_46'): 0.703034,
    (8, 'feature_47'): 0.501402,
    (8, 'feature_48'): -0.215461,
    (8, 'feature_49'): -0.099419,
    (8, 'feature_50'): -0.477729,
    (8, 'feature_51'): 0.055438,
    (8, 'feature_52'): 0.538956,
    (8, 'feature_53'): -0.230489,
    (8, 'feature_54'): -0.230307,
    (8, 'feature_55'): -0.430426,
    (8, 'feature_56'): 1.658538,
    (8, 'feature_57'): 1.340303,
    (8, 'feature_58'): 0.781954,
    (8, 'feature_59'): -0.111282,
    (8, 'feature_60'): 0.075442,
    (8, 'feature_61'): -1.275181,
    (8, 'feature_62'): -0.534438,
    (8, 'feature_63'): -0.383093,
    (8, 'feature_64'): -0.440410,
    (8, 'feature_65'): 0.003965,
    (8, 'feature_66'): 2.493197,
    (8, 'feature_67'): -0.009849,
    (8, 'feature_68'): -0.171259,
    (8, 'feature_69'): -0.212554,
    (8, 'feature_70'): 0.486570,
    (8, 'feature_71'): -0.160993,
    (8, 'feature_72'): -0.053078,
    (8, 'feature_73'): -0.101666,
    (8, 'feature_74'): -0.018268,
    (8, 'feature_75'): 1.715139,
    (8, 'feature_76'): 1.570653,
    (8, 'feature_77'): 0.183638,
    (8, 'feature_78'): 0.409266,
    (9, 'feature_00'): 3.441906,
    (9, 'feature_01'): -0.623610,
    (9, 'feature_02'): 3.003607,
    (9, 'feature_03'): 3.685350,
    (9, 'feature_04'): -1.136381,
    (9, 'feature_05'): 2.149780,
    (9, 'feature_06'): 1.399049,
    (9, 'feature_07'): 0.500464,
    (9, 'feature_08'): -2.281214,
    (9, 'feature_09'): 11.000000,
    (9, 'feature_10'): 7.000000,
    (9, 'feature_11'): 76.000000,
    (9, 'feature_12'): 0.330681,
    (9, 'feature_13'): 0.536635,
    (9, 'feature_14'): 0.391783,
    (9, 'feature_15'): -0.422687,
    (9, 'feature_16'): -0.439299,
    (9, 'feature_17'): -0.594348,
    (9, 'feature_18'): 0.372315,
    (9, 'feature_19'): 4.594866,
    (9, 'feature_20'): -0.284885,
    (9, 'feature_21'): -0.321401,
    (9, 'feature_22'): -0.771719,
    (9, 'feature_23'): 0.148435,
    (9, 'feature_24'): -0.119118,
    (9, 'feature_25'): -1.279223,
    (9, 'feature_26'): 3.115492,
    (9, 'feature_27'): 0.695372,
    (9, 'feature_28'): -0.495012,
    (9, 'feature_29'): -0.652762,
    (9, 'feature_30'): -0.500106,
    (9, 'feature_31'): -0.293870,
    (9, 'feature_32'): 2.478811,
    (9, 'feature_33'): -0.767250,
    (9, 'feature_34'): 2.943006,
    (9, 'feature_35'): 3.496325,
    (9, 'feature_36'): -0.955346,
    (9, 'feature_37'): -1.368477,
    (9, 'feature_38'): -1.994502,
    (9, 'feature_39'): 0.403967,
    (9, 'feature_40'): -0.323898,
    (9, 'feature_41'): 1.615222,
    (9, 'feature_42'): 1.157021,
    (9, 'feature_43'): 0.193363,
    (9, 'feature_44'): 1.032597,
    (9, 'feature_45'): 1.436421,
    (9, 'feature_46'): 1.071212,
    (9, 'feature_47'): 1.093053,
    (9, 'feature_48'): 0.649423,
    (9, 'feature_49'): 0.969502,
    (9, 'feature_50'): 1.116049,
    (9, 'feature_51'): 0.199609,
    (9, 'feature_52'): 1.434663,
    (9, 'feature_53'): 2.000961,
    (9, 'feature_54'): 0.513410,
    (9, 'feature_55'): 0.583316,
    (9, 'feature_56'): 1.977724,
    (9, 'feature_57'): 1.919044,
    (9, 'feature_58'): 1.323140,
    (9, 'feature_59'): 1.401915,
    (9, 'feature_60'): 0.933191,
    (9, 'feature_61'): -1.275181,
    (9, 'feature_62'): -0.257860,
    (9, 'feature_63'): -0.431812,
    (9, 'feature_64'): -0.237907,
    (9, 'feature_65'): -0.159843,
    (9, 'feature_66'): 2.625509,
    (9, 'feature_67'): 0.081545,
    (9, 'feature_68'): -0.032001,
    (9, 'feature_69'): 0.229096,
    (9, 'feature_70'): 0.577537,
    (9, 'feature_71'): 0.920086,
    (9, 'feature_72'): 0.703826,
    (9, 'feature_73'): -0.242988,
    (9, 'feature_74'): -0.067429,
    (9, 'feature_75'): 0.278912,
    (9, 'feature_76'): 0.419284,
    (9, 'feature_77'): -0.110393,
    (9, 'feature_78'): 0.033983,
    (10, 'feature_00'): 3.767523,
    (10, 'feature_01'): -0.474856,
    (10, 'feature_02'): 3.370831,
    (10, 'feature_03'): 2.236662,
    (10, 'feature_04'): -0.834877,
    (10, 'feature_05'): 1.675632,
    (10, 'feature_06'): 0.606404,
    (10, 'feature_07'): 0.619363,
    (10, 'feature_08'): -1.407971,
    (10, 'feature_09'): 42.000000,
    (10, 'feature_10'): 5.000000,
    (10, 'feature_11'): 150.000000,
    (10, 'feature_12'): 0.147042,
    (10, 'feature_13'): -0.036355,
    (10, 'feature_14'): 0.097208,
    (10, 'feature_15'): -0.715645,
    (10, 'feature_16'): -0.555426,
    (10, 'feature_17'): -0.732180,
    (10, 'feature_18'): -2.897641,
    (10, 'feature_19'): 2.473202,
    (10, 'feature_20'): 0.317978,
    (10, 'feature_21'): -0.201837,
    (10, 'feature_22'): 1.049072,
    (10, 'feature_23'): 0.829592,
    (10, 'feature_24'): 0.194130,
    (10, 'feature_25'): -0.891991,
    (10, 'feature_26'): 0.943301,
    (10, 'feature_27'): 0.871294,
    (10, 'feature_28'): -0.255903,
    (10, 'feature_29'): -0.673300,
    (10, 'feature_30'): -0.666683,
    (10, 'feature_31'): -0.214735,
    (10, 'feature_32'): 2.804898,
    (10, 'feature_33'): 0.111845,
    (10, 'feature_34'): 2.935338,
    (10, 'feature_35'): 3.205150,
    (10, 'feature_36'): -0.843832,
    (10, 'feature_37'): -1.199474,
    (10, 'feature_38'): -1.357331,
    (10, 'feature_39'): 0.724948,
    (10, 'feature_40'): -0.600248,
    (10, 'feature_41'): 0.258840,
    (10, 'feature_42'): 0.083229,
    (10, 'feature_43'): 0.171818,
    (10, 'feature_44'): -0.152578,
    (10, 'feature_45'): 0.962033,
    (10, 'feature_46'): 0.675719,
    (10, 'feature_47'): 0.609093,
    (10, 'feature_48'): 0.019199,
    (10, 'feature_49'): 0.191416,
    (10, 'feature_50'): 0.364910,
    (10, 'feature_51'): 0.027559,
    (10, 'feature_52'): 1.227723,
    (10, 'feature_53'): 0.741746,
    (10, 'feature_54'): 0.163903,
    (10, 'feature_55'): 0.131700,
    (10, 'feature_56'): 1.317008,
    (10, 'feature_57'): 1.265226,
    (10, 'feature_58'): 1.081905,
    (10, 'feature_59'): 0.259339,
    (10, 'feature_60'): 0.454517,
    (10, 'feature_61'): -1.275181,
    (10, 'feature_62'): -0.412002,
    (10, 'feature_63'): -0.430983,
    (10, 'feature_64'): -0.391828,
    (10, 'feature_65'): -1.078155,
    (10, 'feature_66'): 2.649467,
    (10, 'feature_67'): -0.171325,
    (10, 'feature_68'): -0.200326,
    (10, 'feature_69'): -0.087586,
    (10, 'feature_70'): 0.275060,
    (10, 'feature_71'): 0.145190,
    (10, 'feature_72'): 0.359837,
    (10, 'feature_73'): -0.334120,
    (10, 'feature_74'): -0.171141,
    (10, 'feature_75'): -0.182068,
    (10, 'feature_76'): -0.053426,
    (10, 'feature_77'): -0.179700,
    (10, 'feature_78'): -0.126067,
    (11, 'feature_00'): 2.997536,
    (11, 'feature_01'): -0.565290,
    (11, 'feature_02'): 3.367284,
    (11, 'feature_03'): 2.728535,
    (11, 'feature_04'): -1.552029,
    (11, 'feature_05'): 1.626889,
    (11, 'feature_06'): 1.850477,
    (11, 'feature_07'): 0.817453,
    (11, 'feature_08'): -2.647647,
    (11, 'feature_09'): 46.000000,
    (11, 'feature_10'): 3.000000,
    (11, 'feature_11'): 261.000000,
    (11, 'feature_12'): 1.596025,
    (11, 'feature_13'): 1.809039,
    (11, 'feature_14'): 1.798146,
    (11, 'feature_15'): -0.728659,
    (11, 'feature_16'): -0.226158,
    (11, 'feature_17'): -0.560625,
    (11, 'feature_18'): -0.537756,
    (11, 'feature_19'): 3.346223,
    (11, 'feature_20'): -0.694198,
    (11, 'feature_21'): -0.206443,
    (11, 'feature_22'): 0.563870,
    (11, 'feature_23'): 0.476391,
    (11, 'feature_24'): -0.526242,
    (11, 'feature_25'): -1.126592,
    (11, 'feature_26'): 0.041071,
    (11, 'feature_27'): 0.053921,
    (11, 'feature_28'): -0.045054,
    (11, 'feature_29'): -0.491220,
    (11, 'feature_30'): -0.537633,
    (11, 'feature_31'): -0.223368,
    (11, 'feature_32'): 3.268108,
    (11, 'feature_33'): -0.702046,
    (11, 'feature_34'): 3.473612,
    (11, 'feature_35'): 2.341969,
    (11, 'feature_36'): -0.847950,
    (11, 'feature_37'): -2.370799,
    (11, 'feature_38'): -2.243412,
    (11, 'feature_39'): 0.802923,
    (11, 'feature_40'): 0.525152,
    (11, 'feature_41'): 1.100384,
    (11, 'feature_42'): 0.409360,
    (11, 'feature_43'): -0.310542,
    (11, 'feature_44'): -0.577615,
    (11, 'feature_45'): 2.204612,
    (11, 'feature_46'): 1.235216,
    (11, 'feature_47'): 2.378664,
    (11, 'feature_48'): 0.091022,
    (11, 'feature_49'): 0.488174,
    (11, 'feature_50'): 1.718315,
    (11, 'feature_51'): 1.362462,
    (11, 'feature_52'): 0.545779,
    (11, 'feature_53'): 0.819499,
    (11, 'feature_54'): -0.151533,
    (11, 'feature_55'): -0.481445,
    (11, 'feature_56'): 2.001767,
    (11, 'feature_57'): 1.342734,
    (11, 'feature_58'): 4.117878,
    (11, 'feature_59'): 0.645626,
    (11, 'feature_60'): 0.627561,
    (11, 'feature_61'): -1.275181,
    (11, 'feature_62'): -0.329699,
    (11, 'feature_63'): -0.110361,
    (11, 'feature_64'): -0.404492,
    (11, 'feature_65'): -0.316889,
    (11, 'feature_66'): 3.211047,
    (11, 'feature_67'): 0.937190,
    (11, 'feature_68'): 1.555800,
    (11, 'feature_69'): 1.499583,
    (11, 'feature_70'): 2.861647,
    (11, 'feature_71'): 2.452162,
    (11, 'feature_72'): 2.349124,
    (11, 'feature_73'): -0.061972,
    (11, 'feature_74'): 0.100150,
    (11, 'feature_75'): 1.591393,
    (11, 'feature_76'): 2.663491,
    (11, 'feature_77'): 0.183892,
    (11, 'feature_78'): 0.303436,
    (12, 'feature_00'): 3.229622,
    (12, 'feature_01'): -0.957581,
    (12, 'feature_02'): 3.281682,
    (12, 'feature_03'): 3.195697,
    (12, 'feature_04'): -1.522871,
    (12, 'feature_05'): 1.130756,
    (12, 'feature_06'): 1.016201,
    (12, 'feature_07'): 0.462710,
    (12, 'feature_08'): -1.588595,
    (12, 'feature_09'): 70.000000,
    (12, 'feature_10'): 7.000000,
    (12, 'feature_11'): 410.000000,
    (12, 'feature_12'): 0.304202,
    (12, 'feature_13'): 1.000073,
    (12, 'feature_14'): 0.558641,
    (12, 'feature_15'): -0.504202,
    (12, 'feature_16'): -0.216303,
    (12, 'feature_17'): -0.425039,
    (12, 'feature_18'): -1.635622,
    (12, 'feature_19'): 4.326123,
    (12, 'feature_20'): -0.730119,
    (12, 'feature_21'): 0.121312,
    (12, 'feature_22'): 0.333169,
    (12, 'feature_23'): 1.117514,
    (12, 'feature_24'): 0.772262,
    (12, 'feature_25'): 0.117220,
    (12, 'feature_26'): 0.643476,
    (12, 'feature_27'): -1.473181,
    (12, 'feature_28'): -1.702426,
    (12, 'feature_29'): -0.185288,
    (12, 'feature_30'): -0.453919,
    (12, 'feature_31'): 0.087466,
    (12, 'feature_32'): 2.977635,
    (12, 'feature_33'): -0.484802,
    (12, 'feature_34'): 3.312300,
    (12, 'feature_35'): 2.908534,
    (12, 'feature_36'): -0.857011,
    (12, 'feature_37'): -2.025003,
    (12, 'feature_38'): -1.861683,
    (12, 'feature_39'): 0.851269,
    (12, 'feature_40'): 1.069923,
    (12, 'feature_41'): 0.810555,
    (12, 'feature_42'): 0.483406,
    (12, 'feature_43'): -0.069422,
    (12, 'feature_44'): 0.290318,
    (12, 'feature_45'): 1.128873,
    (12, 'feature_46'): 1.355186,
    (12, 'feature_47'): 1.076289,
    (12, 'feature_48'): 1.025909,
    (12, 'feature_49'): 1.033979,
    (12, 'feature_50'): 1.535069,
    (12, 'feature_51'): 0.829627,
    (12, 'feature_52'): 1.342368,
    (12, 'feature_53'): 1.376789,
    (12, 'feature_54'): -0.503570,
    (12, 'feature_55'): -0.014389,
    (12, 'feature_56'): 1.391297,
    (12, 'feature_57'): 1.719828,
    (12, 'feature_58'): 1.093974,
    (12, 'feature_59'): 1.162750,
    (12, 'feature_60'): 0.874436,
    (12, 'feature_61'): -1.275181,
    (12, 'feature_62'): -0.331122,
    (12, 'feature_63'): -0.229233,
    (12, 'feature_64'): -0.268038,
    (12, 'feature_65'): -0.357063,
    (12, 'feature_66'): 4.432940,
    (12, 'feature_67'): -0.117947,
    (12, 'feature_68'): 0.288318,
    (12, 'feature_69'): 0.264302,
    (12, 'feature_70'): 0.442789,
    (12, 'feature_71'): 1.618170,
    (12, 'feature_72'): 1.274666,
    (12, 'feature_73'): -0.301783,
    (12, 'feature_74'): -0.153018,
    (12, 'feature_75'): 0.080975,
    (12, 'feature_76'): 0.313266,
    (12, 'feature_77'): -0.108002,
    (12, 'feature_78'): 0.042109,
    (13, 'feature_00'): 3.156799,
    (13, 'feature_01'): -0.617133,
    (13, 'feature_02'): 2.860576,
    (13, 'feature_03'): 3.609724,
    (13, 'feature_04'): -0.589447,
    (13, 'feature_05'): 1.351276,
    (13, 'feature_06'): 0.846722,
    (13, 'feature_07'): 0.401311,
    (13, 'feature_08'): -1.284864,
    (13, 'feature_09'): 50.000000,
    (13, 'feature_10'): 1.000000,
    (13, 'feature_11'): 522.000000,
    (13, 'feature_12'): 0.901380,
    (13, 'feature_13'): 0.148324,
    (13, 'feature_14'): 0.588572,
    (13, 'feature_15'): -0.781552,
    (13, 'feature_16'): -0.487297,
    (13, 'feature_17'): -0.459563,
    (13, 'feature_18'): -2.169817,
    (13, 'feature_19'): 1.481533,
    (13, 'feature_20'): 0.273780,
    (13, 'feature_21'): -0.067842,
    (13, 'feature_22'): 0.150324,
    (13, 'feature_23'): 0.344110,
    (13, 'feature_24'): 1.250794,
    (13, 'feature_25'): 0.824879,
    (13, 'feature_26'): 2.379274,
    (13, 'feature_27'): -0.815149,
    (13, 'feature_28'): -1.612664,
    (13, 'feature_29'): -0.528744,
    (13, 'feature_30'): -0.688517,
    (13, 'feature_31'): -0.038668,
    (13, 'feature_32'): 3.047245,
    (13, 'feature_33'): -0.620645,
    (13, 'feature_34'): 2.463547,
    (13, 'feature_35'): 3.169645,
    (13, 'feature_36'): -0.716500,
    (13, 'feature_37'): -2.003288,
    (13, 'feature_38'): -2.262764,
    (13, 'feature_39'): 0.529539,
    (13, 'feature_40'): -0.358239,
    (13, 'feature_41'): 1.236567,
    (13, 'feature_42'): -0.620180,
    (13, 'feature_43'): -0.138171,
    (13, 'feature_44'): 0.623873,
    (13, 'feature_45'): 1.370310,
    (13, 'feature_46'): 0.641967,
    (13, 'feature_47'): 0.434191,
    (13, 'feature_48'): 0.213843,
    (13, 'feature_49'): 0.260632,
    (13, 'feature_50'): 1.323941,
    (13, 'feature_51'): -0.040353,
    (13, 'feature_52'): 1.256053,
    (13, 'feature_53'): 0.086457,
    (13, 'feature_54'): 0.219090,
    (13, 'feature_55'): 0.914972,
    (13, 'feature_56'): 1.675518,
    (13, 'feature_57'): 0.574600,
    (13, 'feature_58'): 0.744420,
    (13, 'feature_59'): 0.148971,
    (13, 'feature_60'): 0.253813,
    (13, 'feature_61'): -1.275181,
    (13, 'feature_62'): -0.432012,
    (13, 'feature_63'): -0.339511,
    (13, 'feature_64'): -0.283512,
    (13, 'feature_65'): -2.111955,
    (13, 'feature_66'): 2.289829,
    (13, 'feature_67'): 0.564919,
    (13, 'feature_68'): 0.087716,
    (13, 'feature_69'): 0.510993,
    (13, 'feature_70'): 0.501141,
    (13, 'feature_71'): 0.123650,
    (13, 'feature_72'): 0.489926,
    (13, 'feature_73'): -0.285390,
    (13, 'feature_74'): -0.058262,
    (13, 'feature_75'): -0.050342,
    (13, 'feature_76'): 0.186161,
    (13, 'feature_77'): -0.107555,
    (13, 'feature_78'): 0.078949,
    (14, 'feature_00'): 2.772789,
    (14, 'feature_01'): -0.273481,
    (14, 'feature_02'): 3.379932,
    (14, 'feature_03'): 3.164012,
    (14, 'feature_04'): 0.067493,
    (14, 'feature_05'): 1.533186,
    (14, 'feature_06'): 1.097498,
    (14, 'feature_07'): 0.557751,
    (14, 'feature_08'): -1.515057,
    (14, 'feature_09'): 44.000000,
    (14, 'feature_10'): 3.000000,
    (14, 'feature_11'): 16.000000,
    (14, 'feature_12'): 0.466402,
    (14, 'feature_13'): 0.210745,
    (14, 'feature_14'): 0.426430,
    (14, 'feature_15'): -0.579891,
    (14, 'feature_16'): -0.668052,
    (14, 'feature_17'): -0.597626,
    (14, 'feature_18'): 0.924093,
    (14, 'feature_19'): 2.349615,
    (14, 'feature_20'): -0.151023,
    (14, 'feature_21'): -0.274223,
    (14, 'feature_22'): -0.543829,
    (14, 'feature_23'): -0.162202,
    (14, 'feature_24'): 0.386456,
    (14, 'feature_25'): -0.873935,
    (14, 'feature_26'): -0.609433,
    (14, 'feature_27'): -0.351026,
    (14, 'feature_28'): -0.455671,
    (14, 'feature_29'): -0.524518,
    (14, 'feature_30'): -0.372637,
    (14, 'feature_31'): -0.295973,
    (14, 'feature_32'): 2.781836,
    (14, 'feature_33'): -0.438187,
    (14, 'feature_34'): 2.988264,
    (14, 'feature_35'): 2.391832,
    (14, 'feature_36'): -1.149521,
    (14, 'feature_37'): -0.669777,
    (14, 'feature_38'): -0.739268,
    (14, 'feature_39'): 0.561504,
    (14, 'feature_40'): 0.356492,
    (14, 'feature_41'): 0.909552,
    (14, 'feature_42'): -0.423071,
    (14, 'feature_43'): -0.024016,
    (14, 'feature_44'): 0.295929,
    (14, 'feature_45'): 1.550539,
    (14, 'feature_46'): 0.494008,
    (14, 'feature_47'): 0.981564,
    (14, 'feature_48'): 0.154953,
    (14, 'feature_49'): 0.256084,
    (14, 'feature_50'): 1.386451,
    (14, 'feature_51'): 0.395594,
    (14, 'feature_52'): 1.414181,
    (14, 'feature_53'): -0.126589,
    (14, 'feature_54'): 0.316125,
    (14, 'feature_55'): 0.625480,
    (14, 'feature_56'): 2.328427,
    (14, 'feature_57'): 0.594718,
    (14, 'feature_58'): 1.128952,
    (14, 'feature_59'): 0.045380,
    (14, 'feature_60'): 0.467589,
    (14, 'feature_61'): -1.275181,
    (14, 'feature_62'): -0.381602,
    (14, 'feature_63'): -0.322803,
    (14, 'feature_64'): -0.265294,
    (14, 'feature_65'): 0.786176,
    (14, 'feature_66'): 2.801067,
    (14, 'feature_67'): 0.443800,
    (14, 'feature_68'): 0.376064,
    (14, 'feature_69'): 0.717949,
    (14, 'feature_70'): 1.030243,
    (14, 'feature_71'): 0.002463,
    (14, 'feature_72'): 0.309285,
    (14, 'feature_73'): 0.104218,
    (14, 'feature_74'): 0.144593,
    (14, 'feature_75'): 0.254098,
    (14, 'feature_76'): 0.293411,
    (14, 'feature_77'): 0.167594,
    (14, 'feature_78'): 0.162861,
    (15, 'feature_00'): 3.558992,
    (15, 'feature_01'): -0.551834,
    (15, 'feature_02'): 3.607774,
    (15, 'feature_03'): 3.215009,
    (15, 'feature_04'): -1.014850,
    (15, 'feature_05'): 1.044046,
    (15, 'feature_06'): 0.986040,
    (15, 'feature_07'): 0.342634,
    (15, 'feature_08'): -1.761179,
    (15, 'feature_09'): 9.000000,
    (15, 'feature_10'): 3.000000,
    (15, 'feature_11'): 25.000000,
    (15, 'feature_12'): 0.514297,
    (15, 'feature_13'): 1.755323,
    (15, 'feature_14'): 0.520711,
    (15, 'feature_15'): -0.749164,
    (15, 'feature_16'): -0.725738,
    (15, 'feature_17'): -0.893297,
    (15, 'feature_18'): -2.813762,
    (15, 'feature_19'): -0.050074,
    (15, 'feature_20'): -0.233193,
    (15, 'feature_21'): -0.257364,
    (15, 'feature_22'): -0.126727,
    (15, 'feature_23'): -0.294687,
    (15, 'feature_24'): 0.533778,
    (15, 'feature_25'): 0.168051,
    (15, 'feature_26'): -1.191492,
    (15, 'feature_27'): -0.569850,
    (15, 'feature_28'): -0.329296,
    (15, 'feature_29'): -0.797958,
    (15, 'feature_30'): -0.860720,
    (15, 'feature_31'): -0.168345,
    (15, 'feature_32'): 2.564062,
    (15, 'feature_33'): -0.415043,
    (15, 'feature_34'): 2.911334,
    (15, 'feature_35'): 2.718266,
    (15, 'feature_36'): -0.509880,
    (15, 'feature_37'): -0.769168,
    (15, 'feature_38'): -0.735152,
    (15, 'feature_39'): 0.060453,
    (15, 'feature_40'): -0.428143,
    (15, 'feature_41'): 0.672834,
    (15, 'feature_42'): 1.064218,
    (15, 'feature_43'): 0.077727,
    (15, 'feature_44'): 0.142611,
    (15, 'feature_45'): -0.170317,
    (15, 'feature_46'): 0.151703,
    (15, 'feature_47'): 0.161516,
    (15, 'feature_48'): 0.227524,
    (15, 'feature_49'): 0.470901,
    (15, 'feature_50'): 0.031650,
    (15, 'feature_51'): 0.013991,
    (15, 'feature_52'): 1.038392,
    (15, 'feature_53'): 2.210000,
    (15, 'feature_54'): 0.808239,
    (15, 'feature_55'): 1.824615,
    (15, 'feature_56'): 0.426689,
    (15, 'feature_57'): 0.357886,
    (15, 'feature_58'): 0.372084,
    (15, 'feature_59'): 0.884594,
    (15, 'feature_60'): 0.720265,
    (15, 'feature_61'): -1.275181,
    (15, 'feature_62'): -0.400447,
    (15, 'feature_63'): -0.277461,
    (15, 'feature_64'): -0.508060,
    (15, 'feature_65'): 1.302884,
    (15, 'feature_66'): 0.177153,
    (15, 'feature_67'): 0.110782,
    (15, 'feature_68'): 0.130171,
    (15, 'feature_69'): 0.168550,
    (15, 'feature_70'): 0.464308,
    (15, 'feature_71'): 5.648359,
    (15, 'feature_72'): 1.503996,
    (15, 'feature_73'): 0.052211,
    (15, 'feature_74'): 0.125100,
    (15, 'feature_75'): -0.117650,
    (15, 'feature_76'): -0.137045,
    (15, 'feature_77'): 0.139669,
    (15, 'feature_78'): 0.169152,
    (16, 'feature_00'): 3.016707,
    (16, 'feature_01'): -0.865127,
    (16, 'feature_02'): 3.582553,
    (16, 'feature_03'): 3.512409,
    (16, 'feature_04'): -0.499617,
    (16, 'feature_05'): 1.369425,
    (16, 'feature_06'): 1.064484,
    (16, 'feature_07'): 0.494004,
    (16, 'feature_08'): -1.733598,
    (16, 'feature_09'): 11.000000,
    (16, 'feature_10'): 7.000000,
    (16, 'feature_11'): 76.000000,
    (16, 'feature_12'): 0.028749,
    (16, 'feature_13'): 0.402950,
    (16, 'feature_14'): 0.202204,
    (16, 'feature_15'): -0.903820,
    (16, 'feature_16'): -0.325198,
    (16, 'feature_17'): -0.621586,
    (16, 'feature_18'): 0.021259,
    (16, 'feature_19'): 2.622130,
    (16, 'feature_20'): 1.154341,
    (16, 'feature_21'): -0.073227,
    (16, 'feature_22'): 1.685778,
    (16, 'feature_23'): 1.444645,
    (16, 'feature_24'): 0.758180,
    (16, 'feature_25'): 0.037927,
    (16, 'feature_26'): -0.490630,
    (16, 'feature_27'): 0.783008,
    (16, 'feature_28'): 0.710124,
    (16, 'feature_29'): -0.495942,
    (16, 'feature_30'): -0.709414,
    (16, 'feature_31'): -0.087225,
    (16, 'feature_32'): 2.649727,
    (16, 'feature_33'): -0.028212,
    (16, 'feature_34'): 2.963835,
    (16, 'feature_35'): 3.158931,
    (16, 'feature_36'): -0.046347,
    (16, 'feature_37'): -1.171459,
    (16, 'feature_38'): -2.691438,
    (16, 'feature_39'): 0.920952,
    (16, 'feature_40'): -0.860181,
    (16, 'feature_41'): 1.052381,
    (16, 'feature_42'): -0.183315,
    (16, 'feature_43'): -0.910988,
    (16, 'feature_44'): -0.808299,
    (16, 'feature_45'): 1.560171,
    (16, 'feature_46'): 0.956440,
    (16, 'feature_47'): 0.656989,
    (16, 'feature_48'): -0.702586,
    (16, 'feature_49'): 0.102332,
    (16, 'feature_50'): 1.085761,
    (16, 'feature_51'): -0.992532,
    (16, 'feature_52'): 0.680571,
    (16, 'feature_53'): 0.421920,
    (16, 'feature_54'): -0.803213,
    (16, 'feature_55'): -0.806350,
    (16, 'feature_56'): 1.685056,
    (16, 'feature_57'): 0.850533,
    (16, 'feature_58'): 1.431361,
    (16, 'feature_59'): -1.497998,
    (16, 'feature_60'): 0.262147,
    (16, 'feature_61'): -1.275181,
    (16, 'feature_62'): -0.450615,
    (16, 'feature_63'): -0.305163,
    (16, 'feature_64'): -0.374699,
    (16, 'feature_65'): 0.179566,
    (16, 'feature_66'): 2.696337,
    (16, 'feature_67'): -0.130098,
    (16, 'feature_68'): 0.384851,
    (16, 'feature_69'): 0.035568,
    (16, 'feature_70'): 0.165131,
    (16, 'feature_71'): 0.360541,
    (16, 'feature_72'): 0.347068,
    (16, 'feature_73'): -0.197605,
    (16, 'feature_74'): 0.008010,
    (16, 'feature_75'): -0.079478,
    (16, 'feature_76'): 0.043519,
    (16, 'feature_77'): 0.007909,
    (16, 'feature_78'): 0.219342,
    (17, 'feature_00'): 3.852982,
    (17, 'feature_01'): -0.722106,
    (17, 'feature_02'): 2.966254,
    (17, 'feature_03'): 3.502724,
    (17, 'feature_04'): -0.853670,
    (17, 'feature_05'): 1.481035,
    (17, 'feature_06'): 1.205071,
    (17, 'feature_07'): 0.643708,
    (17, 'feature_08'): -1.978651,
    (17, 'feature_09'): 64.000000,
    (17, 'feature_10'): 6.000000,
    (17, 'feature_11'): 376.000000,
    (17, 'feature_12'): 0.292732,
    (17, 'feature_13'): 0.672684,
    (17, 'feature_14'): 0.412609,
    (17, 'feature_15'): -0.888165,
    (17, 'feature_16'): -0.496488,
    (17, 'feature_17'): -0.494074,
    (17, 'feature_18'): -0.594986,
    (17, 'feature_19'): 3.395260,
    (17, 'feature_20'): 0.296760,
    (17, 'feature_21'): -0.149440,
    (17, 'feature_22'): 2.178973,
    (17, 'feature_23'): 2.004284,
    (17, 'feature_24'): 0.352751,
    (17, 'feature_25'): -0.826058,
    (17, 'feature_26'): 0.260487,
    (17, 'feature_27'): 1.319281,
    (17, 'feature_28'): 1.226046,
    (17, 'feature_29'): -0.568684,
    (17, 'feature_30'): -0.521869,
    (17, 'feature_31'): -0.206032,
    (17, 'feature_32'): 2.890348,
    (17, 'feature_33'): -0.604204,
    (17, 'feature_34'): 3.377552,
    (17, 'feature_35'): 2.849094,
    (17, 'feature_36'): -0.884936,
    (17, 'feature_37'): -1.167338,
    (17, 'feature_38'): -1.348824,
    (17, 'feature_39'): 0.286647,
    (17, 'feature_40'): 0.694392,
    (17, 'feature_41'): -0.092289,
    (17, 'feature_42'): 0.255401,
    (17, 'feature_43'): -0.233999,
    (17, 'feature_44'): -0.547776,
    (17, 'feature_45'): 1.131330,
    (17, 'feature_46'): 1.607601,
    (17, 'feature_47'): 0.902082,
    (17, 'feature_48'): -0.207839,
    (17, 'feature_49'): -0.005586,
    (17, 'feature_50'): 0.458556,
    (17, 'feature_51'): -0.823130,
    (17, 'feature_52'): -1.967599,
    (17, 'feature_53'): -0.278163,
    (17, 'feature_54'): 0.364664,
    (17, 'feature_55'): -0.121375,
    (17, 'feature_56'): 1.176575,
    (17, 'feature_57'): 1.094112,
    (17, 'feature_58'): 1.483010,
    (17, 'feature_59'): -0.495717,
    (17, 'feature_60'): -0.004854,
    (17, 'feature_61'): -1.275181,
    (17, 'feature_62'): -0.520475,
    (17, 'feature_63'): -0.288892,
    (17, 'feature_64'): -0.354650,
    (17, 'feature_65'): -1.087096,
    (17, 'feature_66'): 2.079908,
    (17, 'feature_67'): 0.345653,
    (17, 'feature_68'): 0.443340,
    (17, 'feature_69'): 0.310842,
    (17, 'feature_70'): 0.917355,
    (17, 'feature_71'): 0.623741,
    (17, 'feature_72'): 0.346191,
    (17, 'feature_73'): -0.281958,
    (17, 'feature_74'): -0.204550,
    (17, 'feature_75'): -0.241077,
    (17, 'feature_76'): -0.115040,
    (17, 'feature_77'): -0.255413,
    (17, 'feature_78'): -0.172392,
    (18, 'feature_00'): 3.553501,
    (18, 'feature_01'): -0.402203,
    (18, 'feature_02'): 3.463995,
    (18, 'feature_03'): 3.352048,
    (18, 'feature_04'): -0.904168,
    (18, 'feature_05'): 1.228454,
    (18, 'feature_06'): 0.959628,
    (18, 'feature_07'): 0.419548,
    (18, 'feature_08'): -1.105989,
    (18, 'feature_09'): 15.000000,
    (18, 'feature_10'): 1.000000,
    (18, 'feature_11'): 66.000000,
    (18, 'feature_12'): 0.534210,
    (18, 'feature_13'): 0.574701,
    (18, 'feature_14'): 0.924518,
    (18, 'feature_15'): -0.799259,
    (18, 'feature_16'): -0.850187,
    (18, 'feature_17'): -0.613326,
    (18, 'feature_18'): 0.940511,
    (18, 'feature_19'): 3.142272,
    (18, 'feature_20'): 0.354832,
    (18, 'feature_21'): -0.125701,
    (18, 'feature_22'): -0.246438,
    (18, 'feature_23'): -0.014072,
    (18, 'feature_24'): 1.080709,
    (18, 'feature_25'): 1.196221,
    (18, 'feature_26'): 0.196416,
    (18, 'feature_27'): -0.598333,
    (18, 'feature_28'): -0.784532,
    (18, 'feature_29'): -0.588850,
    (18, 'feature_30'): -0.827509,
    (18, 'feature_31'): -0.198609,
    (18, 'feature_32'): 2.594931,
    (18, 'feature_33'): -0.304470,
    (18, 'feature_34'): 2.651196,
    (18, 'feature_35'): 2.978361,
    (18, 'feature_36'): -0.579035,
    (18, 'feature_37'): -0.641002,
    (18, 'feature_38'): -1.016756,
    (18, 'feature_39'): 1.580956,
    (18, 'feature_40'): 0.187635,
    (18, 'feature_41'): 0.749705,
    (18, 'feature_42'): 1.199881,
    (18, 'feature_43'): 0.278136,
    (18, 'feature_44'): 0.837443,
    (18, 'feature_45'): 1.902667,
    (18, 'feature_46'): 1.145723,
    (18, 'feature_47'): 0.858446,
    (18, 'feature_48'): 0.285055,
    (18, 'feature_49'): 0.740774,
    (18, 'feature_50'): 1.074467,
    (18, 'feature_51'): -0.293859,
    (18, 'feature_52'): 1.298835,
    (18, 'feature_53'): 0.894565,
    (18, 'feature_54'): 0.309391,
    (18, 'feature_55'): 1.817863,
    (18, 'feature_56'): 1.950326,
    (18, 'feature_57'): 1.381299,
    (18, 'feature_58'): 1.038182,
    (18, 'feature_59'): 0.818062,
    (18, 'feature_60'): 0.500558,
    (18, 'feature_61'): -1.275181,
    (18, 'feature_62'): -0.436154,
    (18, 'feature_63'): -0.299498,
    (18, 'feature_64'): -0.354645,
    (18, 'feature_65'): 0.113719,
    (18, 'feature_66'): 2.385581,
    (18, 'feature_67'): 0.337017,
    (18, 'feature_68'): 0.812501,
    (18, 'feature_69'): 0.817315,
    (18, 'feature_70'): 0.877488,
    (18, 'feature_71'): 0.495128,
    (18, 'feature_72'): 0.885172,
    (18, 'feature_73'): -0.934562,
    (18, 'feature_74'): -1.409185,
    (18, 'feature_75'): -1.927153,
    (18, 'feature_76'): -1.768026,
    (18, 'feature_77'): -1.750554,
    (18, 'feature_78'): -1.613748,
    (19, 'feature_00'): 3.892014,
    (19, 'feature_01'): -0.617406,
    (19, 'feature_02'): 3.548004,
    (19, 'feature_03'): 3.538568,
    (19, 'feature_04'): -1.126776,
    (19, 'feature_05'): 1.397626,
    (19, 'feature_06'): 0.903944,
    (19, 'feature_07'): 0.412348,
    (19, 'feature_08'): -1.897645,
    (19, 'feature_09'): 4.000000,
    (19, 'feature_10'): 3.000000,
    (19, 'feature_11'): 11.000000,
    (19, 'feature_12'): 0.047128,
    (19, 'feature_13'): 0.417049,
    (19, 'feature_14'): 0.203240,
    (19, 'feature_15'): -0.904385,
    (19, 'feature_16'): -0.788829,
    (19, 'feature_17'): -0.842539,
    (19, 'feature_18'): -1.470274,
    (19, 'feature_19'): 2.427003,
    (19, 'feature_20'): 0.202072,
    (19, 'feature_21'): -0.122558,
    (19, 'feature_22'): 2.056729,
    (19, 'feature_23'): 1.667715,
    (19, 'feature_24'): 0.361405,
    (19, 'feature_25'): -0.357961,
    (19, 'feature_26'): 0.450199,
    (19, 'feature_27'): 2.112297,
    (19, 'feature_28'): 1.630664,
    (19, 'feature_29'): -0.701006,
    (19, 'feature_30'): -0.895150,
    (19, 'feature_31'): -0.125730,
    (19, 'feature_32'): 2.583360,
    (19, 'feature_33'): 0.277304,
    (19, 'feature_34'): 3.062153,
    (19, 'feature_35'): 3.438560,
    (19, 'feature_36'): -0.899352,
    (19, 'feature_37'): -1.122738,
    (19, 'feature_38'): -1.385075,
    (19, 'feature_39'): 0.282442,
    (19, 'feature_40'): 0.290849,
    (19, 'feature_41'): 0.336104,
    (19, 'feature_42'): 0.551804,
    (19, 'feature_43'): -0.227460,
    (19, 'feature_44'): 0.029915,
    (19, 'feature_45'): 0.972180,
    (19, 'feature_46'): 1.047850,
    (19, 'feature_47'): 0.377692,
    (19, 'feature_48'): 0.316518,
    (19, 'feature_49'): 0.246209,
    (19, 'feature_50'): 0.510186,
    (19, 'feature_51'): 0.814205,
    (19, 'feature_52'): 0.562772,
    (19, 'feature_53'): 1.087742,
    (19, 'feature_54'): 0.841740,
    (19, 'feature_55'): 0.821135,
    (19, 'feature_56'): 1.417423,
    (19, 'feature_57'): 1.737492,
    (19, 'feature_58'): 0.676162,
    (19, 'feature_59'): 0.902095,
    (19, 'feature_60'): 0.544779,
    (19, 'feature_61'): -1.275181,
    (19, 'feature_62'): -0.591809,
    (19, 'feature_63'): -0.407354,
    (19, 'feature_64'): -0.434421,
    (19, 'feature_65'): -1.103402,
    (19, 'feature_66'): 2.544557,
    (19, 'feature_67'): -0.110287,
    (19, 'feature_68'): -0.056290,
    (19, 'feature_69'): 0.021829,
    (19, 'feature_70'): 0.326753,
    (19, 'feature_71'): 0.991337,
    (19, 'feature_72'): 0.374782,
    (19, 'feature_73'): -0.135440,
    (19, 'feature_74'): 0.001618,
    (19, 'feature_75'): -0.241455,
    (19, 'feature_76'): -0.066321,
    (19, 'feature_77'): -0.156141,
    (19, 'feature_78'): -0.004084,
    (20, 'feature_00'): 3.808794,
    (20, 'feature_01'): -0.403028,
    (20, 'feature_02'): 3.261562,
    (20, 'feature_03'): 3.259307,
    (20, 'feature_04'): -0.491069,
    (20, 'feature_05'): 1.027770,
    (20, 'feature_06'): 0.551902,
    (20, 'feature_07'): 0.341068,
    (20, 'feature_08'): -0.869327,
    (20, 'feature_09'): 30.000000,
    (20, 'feature_10'): 10.000000,
    (20, 'feature_11'): 50.000000,
    (20, 'feature_12'): 0.020019,
    (20, 'feature_13'): -0.146628,
    (20, 'feature_14'): -0.131021,
    (20, 'feature_15'): -0.762020,
    (20, 'feature_16'): -0.677525,
    (20, 'feature_17'): -0.643760,
    (20, 'feature_18'): 0.107339,
    (20, 'feature_19'): 2.086795,
    (20, 'feature_20'): -0.108531,
    (20, 'feature_21'): -0.251157,
    (20, 'feature_22'): -1.084780,
    (20, 'feature_23'): -0.871433,
    (20, 'feature_24'): 2.232605,
    (20, 'feature_25'): 1.478186,
    (20, 'feature_26'): 1.203126,
    (20, 'feature_27'): -1.704663,
    (20, 'feature_28'): -2.249491,
    (20, 'feature_29'): -0.728319,
    (20, 'feature_30'): -0.965175,
    (20, 'feature_31'): -0.177198,
    (20, 'feature_32'): 2.990630,
    (20, 'feature_33'): 0.196139,
    (20, 'feature_34'): 2.780401,
    (20, 'feature_35'): 3.154098,
    (20, 'feature_36'): -1.254028,
    (20, 'feature_37'): -1.082815,
    (20, 'feature_38'): -0.895901,
    (20, 'feature_39'): -0.422808,
    (20, 'feature_40'): 0.606121,
    (20, 'feature_41'): 0.328053,
    (20, 'feature_42'): -0.396862,
    (20, 'feature_43'): 0.262499,
    (20, 'feature_44'): -0.095472,
    (20, 'feature_45'): 1.024133,
    (20, 'feature_46'): 0.538960,
    (20, 'feature_47'): 0.320159,
    (20, 'feature_48'): 0.454801,
    (20, 'feature_49'): 0.211881,
    (20, 'feature_50'): 0.207263,
    (20, 'feature_51'): -0.342525,
    (20, 'feature_52'): -0.093306,
    (20, 'feature_53'): 0.448433,
    (20, 'feature_54'): 0.830925,
    (20, 'feature_55'): 1.012235,
    (20, 'feature_56'): 0.650774,
    (20, 'feature_57'): 1.064805,
    (20, 'feature_58'): 0.661359,
    (20, 'feature_59'): 0.303078,
    (20, 'feature_60'): 0.386282,
    (20, 'feature_61'): -1.275181,
    (20, 'feature_62'): -0.376614,
    (20, 'feature_63'): -0.409459,
    (20, 'feature_64'): -0.516712,
    (20, 'feature_65'): -0.256388,
    (20, 'feature_66'): 2.115470,
    (20, 'feature_67'): -0.111829,
    (20, 'feature_68'): -0.147829,
    (20, 'feature_69'): -0.202658,
    (20, 'feature_70'): 0.200615,
    (20, 'feature_71'): -0.197231,
    (20, 'feature_72'): 0.046490,
    (20, 'feature_73'): -0.166104,
    (20, 'feature_74'): -0.012252,
    (20, 'feature_75'): -0.792500,
    (20, 'feature_76'): -0.709068,
    (20, 'feature_77'): -0.646469,
    (20, 'feature_78'): -0.541100,
    (21, 'feature_00'): 3.339161,
    (21, 'feature_01'): -0.694396,
    (21, 'feature_02'): 3.114113,
    (21, 'feature_03'): 3.353838,
    (21, 'feature_04'): -0.461255,
    (21, 'feature_05'): 1.296702,
    (21, 'feature_06'): 0.837965,
    (21, 'feature_07'): 0.412410,
    (21, 'feature_08'): -1.642955,
    (21, 'feature_09'): 34.000000,
    (21, 'feature_10'): 4.000000,
    (21, 'feature_11'): 214.000000,
    (21, 'feature_12'): 2.946527,
    (21, 'feature_13'): 3.971915,
    (21, 'feature_14'): 4.415190,
    (21, 'feature_15'): -0.707123,
    (21, 'feature_16'): -0.512551,
    (21, 'feature_17'): -0.659628,
    (21, 'feature_18'): 1.758790,
    (21, 'feature_19'): 2.813230,
    (21, 'feature_20'): -0.728373,
    (21, 'feature_21'): -0.184218,
    (21, 'feature_22'): -0.958971,
    (21, 'feature_23'): -0.416729,
    (21, 'feature_24'): 0.841932,
    (21, 'feature_25'): 0.830480,
    (21, 'feature_26'): -0.720222,
    (21, 'feature_27'): -1.882254,
    (21, 'feature_28'): -1.536294,
    (21, 'feature_29'): -0.429984,
    (21, 'feature_30'): -0.535485,
    (21, 'feature_31'): -0.129999,
    (21, 'feature_32'): 3.400769,
    (21, 'feature_33'): 0.030592,
    (21, 'feature_34'): 3.605380,
    (21, 'feature_35'): 2.900841,
    (21, 'feature_36'): 0.652732,
    (21, 'feature_37'): -0.304481,
    (21, 'feature_38'): -0.383081,
    (21, 'feature_39'): 0.947629,
    (21, 'feature_40'): -0.869843,
    (21, 'feature_41'): -0.011793,
    (21, 'feature_42'): -1.552853,
    (21, 'feature_43'): -1.045136,
    (21, 'feature_44'): -1.140142,
    (21, 'feature_45'): -0.823926,
    (21, 'feature_46'): -0.316140,
    (21, 'feature_47'): -0.177397,
    (21, 'feature_48'): -0.918603,
    (21, 'feature_49'): -1.752442,
    (21, 'feature_50'): 0.871940,
    (21, 'feature_51'): -0.627676,
    (21, 'feature_52'): -2.018460,
    (21, 'feature_53'): -1.923866,
    (21, 'feature_54'): -0.904219,
    (21, 'feature_55'): -2.129415,
    (21, 'feature_56'): -0.254518,
    (21, 'feature_57'): -0.123928,
    (21, 'feature_58'): -0.551343,
    (21, 'feature_59'): -1.714148,
    (21, 'feature_60'): -2.551266,
    (21, 'feature_61'): -1.275181,
    (21, 'feature_62'): -0.441316,
    (21, 'feature_63'): -0.308639,
    (21, 'feature_64'): -0.261006,
    (21, 'feature_65'): 1.036136,
    (21, 'feature_66'): 3.036869,
    (21, 'feature_67'): 1.217499,
    (21, 'feature_68'): 1.971134,
    (21, 'feature_69'): 2.107481,
    (21, 'feature_70'): 4.162541,
    (21, 'feature_71'): 4.204497,
    (21, 'feature_72'): 5.137740,
    (21, 'feature_73'): -0.183668,
    (21, 'feature_74'): -0.181644,
    (21, 'feature_75'): 0.669623,
    (21, 'feature_76'): 0.648278,
    (21, 'feature_77'): -0.053443,
    (21, 'feature_78'): -0.015080,
    (22, 'feature_00'): 3.626026,
    (22, 'feature_01'): -0.328632,
    (22, 'feature_02'): 3.467623,
    (22, 'feature_03'): 3.896522,
    (22, 'feature_04'): -0.835238,
    (22, 'feature_05'): 1.282962,
    (22, 'feature_06'): 0.977241,
    (22, 'feature_07'): 0.598282,
    (22, 'feature_08'): -2.085168,
    (22, 'feature_09'): 30.000000,
    (22, 'feature_10'): 10.000000,
    (22, 'feature_11'): 13.000000,
    (22, 'feature_12'): 1.918857,
    (22, 'feature_13'): 1.124489,
    (22, 'feature_14'): 1.714180,
    (22, 'feature_15'): -0.517898,
    (22, 'feature_16'): -0.200556,
    (22, 'feature_17'): -0.495334,
    (22, 'feature_18'): 1.569122,
    (22, 'feature_19'): -0.405695,
    (22, 'feature_20'): -0.047656,
    (22, 'feature_21'): -0.228701,
    (22, 'feature_22'): -0.941390,
    (22, 'feature_23'): -0.218096,
    (22, 'feature_24'): 0.542929,
    (22, 'feature_25'): -0.756930,
    (22, 'feature_26'): 0.478320,
    (22, 'feature_27'): -0.689695,
    (22, 'feature_28'): -0.853201,
    (22, 'feature_29'): -0.651745,
    (22, 'feature_30'): -0.608505,
    (22, 'feature_31'): -0.179864,
    (22, 'feature_32'): 3.349580,
    (22, 'feature_33'): -0.732265,
    (22, 'feature_34'): 2.946932,
    (22, 'feature_35'): 3.633484,
    (22, 'feature_36'): 1.072098,
    (22, 'feature_37'): 1.708724,
    (22, 'feature_38'): 1.476705,
    (22, 'feature_39'): 1.046034,
    (22, 'feature_40'): 0.760835,
    (22, 'feature_41'): 0.540184,
    (22, 'feature_42'): -1.049651,
    (22, 'feature_43'): 0.123524,
    (22, 'feature_44'): -2.332603,
    (22, 'feature_45'): 1.346619,
    (22, 'feature_46'): -1.181058,
    (22, 'feature_47'): 0.921537,
    (22, 'feature_48'): -0.065713,
    (22, 'feature_49'): -0.250358,
    (22, 'feature_50'): 1.668677,
    (22, 'feature_51'): -0.391143,
    (22, 'feature_52'): 2.161196,
    (22, 'feature_53'): -0.767696,
    (22, 'feature_54'): -0.720970,
    (22, 'feature_55'): -2.045370,
    (22, 'feature_56'): 0.858700,
    (22, 'feature_57'): -1.326538,
    (22, 'feature_58'): 1.150751,
    (22, 'feature_59'): 0.236381,
    (22, 'feature_60'): -0.584966,
    (22, 'feature_61'): -1.275181,
    (22, 'feature_62'): -0.333555,
    (22, 'feature_63'): -0.099907,
    (22, 'feature_64'): -0.270715,
    (22, 'feature_65'): 1.845558,
    (22, 'feature_66'): -1.209789,
    (22, 'feature_67'): 1.184645,
    (22, 'feature_68'): 1.209521,
    (22, 'feature_69'): 1.475207,
    (22, 'feature_70'): 1.443844,
    (22, 'feature_71'): 0.807368,
    (22, 'feature_72'): 0.825894,
    (22, 'feature_73'): 0.135860,
    (22, 'feature_74'): 0.031522,
    (22, 'feature_75'): 0.510229,
    (22, 'feature_76'): 0.600041,
    (22, 'feature_77'): 0.645749,
    (22, 'feature_78'): 0.511528,
    (23, 'feature_00'): 3.218179,
    (23, 'feature_01'): -1.120938,
    (23, 'feature_02'): 3.572203,
    (23, 'feature_03'): 3.462245,
    (23, 'feature_04'): -1.347046,
    (23, 'feature_05'): 1.143838,
    (23, 'feature_06'): 1.290984,
    (23, 'feature_07'): 0.744361,
    (23, 'feature_08'): -1.823671,
    (23, 'feature_09'): 34.000000,
    (23, 'feature_10'): 4.000000,
    (23, 'feature_11'): 214.000000,
    (23, 'feature_12'): 0.673427,
    (23, 'feature_13'): 0.811712,
    (23, 'feature_14'): 0.440188,
    (23, 'feature_15'): -0.560317,
    (23, 'feature_16'): -0.504328,
    (23, 'feature_17'): -0.600915,
    (23, 'feature_18'): -2.630404,
    (23, 'feature_19'): 2.887101,
    (23, 'feature_20'): -0.305119,
    (23, 'feature_21'): -0.213463,
    (23, 'feature_22'): 0.024690,
    (23, 'feature_23'): 0.531535,
    (23, 'feature_24'): 0.177630,
    (23, 'feature_25'): -0.768967,
    (23, 'feature_26'): 2.124574,
    (23, 'feature_27'): 1.103188,
    (23, 'feature_28'): 0.346348,
    (23, 'feature_29'): -0.586383,
    (23, 'feature_30'): -0.581839,
    (23, 'feature_31'): -0.294406,
    (23, 'feature_32'): 2.277529,
    (23, 'feature_33'): -0.459137,
    (23, 'feature_34'): 3.251171,
    (23, 'feature_35'): 3.129730,
    (23, 'feature_36'): -0.042515,
    (23, 'feature_37'): -1.690262,
    (23, 'feature_38'): -1.990964,
    (23, 'feature_39'): -1.078406,
    (23, 'feature_40'): -0.372839,
    (23, 'feature_41'): -0.407150,
    (23, 'feature_42'): -1.690181,
    (23, 'feature_43'): -0.512466,
    (23, 'feature_44'): -0.247498,
    (23, 'feature_45'): 0.791622,
    (23, 'feature_46'): -0.949929,
    (23, 'feature_47'): -0.359615,
    (23, 'feature_48'): -1.411140,
    (23, 'feature_49'): -1.240995,
    (23, 'feature_50'): -0.795812,
    (23, 'feature_51'): -0.012074,
    (23, 'feature_52'): -0.859957,
    (23, 'feature_53'): -1.211779,
    (23, 'feature_54'): -0.849431,
    (23, 'feature_55'): -1.204939,
    (23, 'feature_56'): 1.507399,
    (23, 'feature_57'): -0.399787,
    (23, 'feature_58'): -1.342864,
    (23, 'feature_59'): -2.637032,
    (23, 'feature_60'): -1.938576,
    (23, 'feature_61'): -1.275181,
    (23, 'feature_62'): -0.385556,
    (23, 'feature_63'): -0.254302,
    (23, 'feature_64'): -0.281283,
    (23, 'feature_65'): -1.553077,
    (23, 'feature_66'): 1.948518,
    (23, 'feature_67'): 0.329895,
    (23, 'feature_68'): 1.208419,
    (23, 'feature_69'): 0.609876,
    (23, 'feature_70'): 0.740569,
    (23, 'feature_71'): 0.094720,
    (23, 'feature_72'): 0.211450,
    (23, 'feature_73'): -0.144745,
    (23, 'feature_74'): 0.003422,
    (23, 'feature_75'): -0.088469,
    (23, 'feature_76'): 0.000102,
    (23, 'feature_77'): -0.093840,
    (23, 'feature_78'): 0.031941,
    (24, 'feature_00'): 3.369400,
    (24, 'feature_01'): -0.896020,
    (24, 'feature_02'): 2.976831,
    (24, 'feature_03'): 2.896805,
    (24, 'feature_04'): -1.235957,
    (24, 'feature_05'): 0.941199,
    (24, 'feature_06'): 0.897569,
    (24, 'feature_07'): 0.528985,
    (24, 'feature_08'): -0.950222,
    (24, 'feature_09'): 14.000000,
    (24, 'feature_10'): 10.000000,
    (24, 'feature_11'): 40.000000,
    (24, 'feature_12'): 0.634756,
    (24, 'feature_13'): 1.580174,
    (24, 'feature_14'): 0.517867,
    (24, 'feature_15'): -0.724432,
    (24, 'feature_16'): -0.520350,
    (24, 'feature_17'): -0.670722,
    (24, 'feature_18'): -3.276833,
    (24, 'feature_19'): 3.592456,
    (24, 'feature_20'): 0.552532,
    (24, 'feature_21'): -0.177095,
    (24, 'feature_22'): -0.303728,
    (24, 'feature_23'): -0.016669,
    (24, 'feature_24'): 1.386660,
    (24, 'feature_25'): 1.142539,
    (24, 'feature_26'): -0.933921,
    (24, 'feature_27'): -1.228485,
    (24, 'feature_28'): -0.635265,
    (24, 'feature_29'): -0.652440,
    (24, 'feature_30'): -0.826935,
    (24, 'feature_31'): -0.203400,
    (24, 'feature_32'): 1.998022,
    (24, 'feature_33'): -0.046076,
    (24, 'feature_34'): 3.284066,
    (24, 'feature_35'): 2.591913,
    (24, 'feature_36'): 1.650165,
    (24, 'feature_37'): -0.916579,
    (24, 'feature_38'): -0.951305,
    (24, 'feature_39'): 0.207541,
    (24, 'feature_40'): 0.906780,
    (24, 'feature_41'): 0.001241,
    (24, 'feature_42'): -0.904963,
    (24, 'feature_43'): 0.021828,
    (24, 'feature_44'): -1.211371,
    (24, 'feature_45'): 0.668695,
    (24, 'feature_46'): 0.010264,
    (24, 'feature_47'): 0.318801,
    (24, 'feature_48'): -0.867844,
    (24, 'feature_49'): -0.551990,
    (24, 'feature_50'): -1.052163,
    (24, 'feature_51'): -0.368106,
    (24, 'feature_52'): -1.688037,
    (24, 'feature_53'): -1.606468,
    (24, 'feature_54'): -0.936930,
    (24, 'feature_55'): -1.548406,
    (24, 'feature_56'): 0.735821,
    (24, 'feature_57'): -0.179077,
    (24, 'feature_58'): 0.262374,
    (24, 'feature_59'): -1.512465,
    (24, 'feature_60'): -2.115434,
    (24, 'feature_61'): -1.275181,
    (24, 'feature_62'): -0.429187,
    (24, 'feature_63'): -0.217958,
    (24, 'feature_64'): -0.245400,
    (24, 'feature_65'): -1.617880,
    (24, 'feature_66'): 3.198682,
    (24, 'feature_67'): 0.030523,
    (24, 'feature_68'): 1.016238,
    (24, 'feature_69'): 0.179699,
    (24, 'feature_70'): 1.110562,
    (24, 'feature_71'): 0.883553,
    (24, 'feature_72'): 0.883279,
    (24, 'feature_73'): -0.106379,
    (24, 'feature_74'): 0.065068,
    (24, 'feature_75'): 0.167059,
    (24, 'feature_76'): 0.319932,
    (24, 'feature_77'): 0.022398,
    (24, 'feature_78'): 0.127624,
    (25, 'feature_00'): 3.341342,
    (25, 'feature_01'): -0.453349,
    (25, 'feature_02'): 3.729469,
    (25, 'feature_03'): 3.279473,
    (25, 'feature_04'): -0.614240,
    (25, 'feature_05'): 1.740091,
    (25, 'feature_06'): 1.035432,
    (25, 'feature_07'): 0.714457,
    (25, 'feature_08'): -1.407195,
    (25, 'feature_09'): 57.000000,
    (25, 'feature_10'): 1.000000,
    (25, 'feature_11'): 230.000000,
    (25, 'feature_12'): 0.281010,
    (25, 'feature_13'): 0.861553,
    (25, 'feature_14'): 0.640735,
    (25, 'feature_15'): -0.518901,
    (25, 'feature_16'): -0.702521,
    (25, 'feature_17'): -0.830515,
    (25, 'feature_18'): 0.313032,
    (25, 'feature_19'): 3.577157,
    (25, 'feature_20'): 0.483233,
    (25, 'feature_21'): -0.011359,
    (25, 'feature_22'): 0.401670,
    (25, 'feature_23'): 0.371724,
    (25, 'feature_24'): 0.701983,
    (25, 'feature_25'): 0.230744,
    (25, 'feature_26'): -0.628254,
    (25, 'feature_27'): -1.116295,
    (25, 'feature_28'): -0.524590,
    (25, 'feature_29'): -0.721156,
    (25, 'feature_30'): -0.562719,
    (25, 'feature_31'): -0.015762,
    (25, 'feature_32'): 2.065979,
    (25, 'feature_33'): -0.801408,
    (25, 'feature_34'): 3.244790,
    (25, 'feature_35'): 2.900053,
    (25, 'feature_36'): 0.211265,
    (25, 'feature_37'): -2.516995,
    (25, 'feature_38'): -1.877949,
    (25, 'feature_39'): 1.623407,
    (25, 'feature_40'): -0.113634,
    (25, 'feature_41'): 1.129041,
    (25, 'feature_42'): 0.067981,
    (25, 'feature_43'): -0.587555,
    (25, 'feature_44'): -0.609755,
    (25, 'feature_45'): 1.782829,
    (25, 'feature_46'): 0.857006,
    (25, 'feature_47'): 0.851362,
    (25, 'feature_48'): -0.475526,
    (25, 'feature_49'): 0.542854,
    (25, 'feature_50'): 1.658271,
    (25, 'feature_51'): -0.262582,
    (25, 'feature_52'): 0.536688,
    (25, 'feature_53'): 0.269574,
    (25, 'feature_54'): -0.131192,
    (25, 'feature_55'): 0.012456,
    (25, 'feature_56'): 2.267947,
    (25, 'feature_57'): 1.248878,
    (25, 'feature_58'): 1.834857,
    (25, 'feature_59'): -0.167829,
    (25, 'feature_60'): 0.708143,
    (25, 'feature_61'): -1.275181,
    (25, 'feature_62'): -0.453446,
    (25, 'feature_63'): -0.333925,
    (25, 'feature_64'): -0.388890,
    (25, 'feature_65'): 0.165648,
    (25, 'feature_66'): 3.056109,
    (25, 'feature_67'): 0.194248,
    (25, 'feature_68'): 0.372493,
    (25, 'feature_69'): 0.375258,
    (25, 'feature_70'): 0.373666,
    (25, 'feature_71'): 1.106526,
    (25, 'feature_72'): 0.629209,
    (25, 'feature_73'): -1.682897,
    (25, 'feature_74'): -1.402045,
    (25, 'feature_75'): -0.671733,
    (25, 'feature_76'): -0.798675,
    (25, 'feature_77'): -1.436295,
    (25, 'feature_78'): -1.247730,
    (26, 'feature_00'): 3.288277,
    (26, 'feature_01'): -0.437545,
    (26, 'feature_02'): 3.301478,
    (26, 'feature_03'): 3.408256,
    (26, 'feature_04'): -0.967216,
    (26, 'feature_05'): 1.280758,
    (26, 'feature_06'): 1.056728,
    (26, 'feature_07'): 0.433609,
    (26, 'feature_08'): -1.237752,
    (26, 'feature_09'): 26.000000,
    (26, 'feature_10'): 12.000000,
    (26, 'feature_11'): 158.000000,
    (26, 'feature_12'): 0.852709,
    (26, 'feature_13'): 0.777193,
    (26, 'feature_14'): 1.211021,
    (26, 'feature_15'): -0.812133,
    (26, 'feature_16'): -0.556338,
    (26, 'feature_17'): -0.782493,
    (26, 'feature_18'): -1.317500,
    (26, 'feature_19'): 3.074778,
    (26, 'feature_20'): -0.767579,
    (26, 'feature_21'): -0.180946,
    (26, 'feature_22'): -0.609799,
    (26, 'feature_23'): -0.345209,
    (26, 'feature_24'): 0.430847,
    (26, 'feature_25'): -0.604465,
    (26, 'feature_26'): -1.322658,
    (26, 'feature_27'): -1.563618,
    (26, 'feature_28'): -0.768010,
    (26, 'feature_29'): -0.935043,
    (26, 'feature_30'): -0.827995,
    (26, 'feature_31'): -0.179455,
    (26, 'feature_32'): 2.758954,
    (26, 'feature_33'): -0.352393,
    (26, 'feature_34'): 2.984877,
    (26, 'feature_35'): 3.059903,
    (26, 'feature_36'): 0.106998,
    (26, 'feature_37'): -1.884484,
    (26, 'feature_38'): -2.201907,
    (26, 'feature_39'): 0.233416,
    (26, 'feature_40'): 0.694147,
    (26, 'feature_41'): 1.262032,
    (26, 'feature_42'): -0.517140,
    (26, 'feature_43'): 0.252979,
    (26, 'feature_44'): -0.576865,
    (26, 'feature_45'): 2.013675,
    (26, 'feature_46'): 0.130457,
    (26, 'feature_47'): 0.561846,
    (26, 'feature_48'): 1.344665,
    (26, 'feature_49'): 0.030801,
    (26, 'feature_50'): 0.502179,
    (26, 'feature_51'): 0.619022,
    (26, 'feature_52'): 1.642458,
    (26, 'feature_53'): -0.915159,
    (26, 'feature_54'): 0.039498,
    (26, 'feature_55'): -0.516107,
    (26, 'feature_56'): 1.297239,
    (26, 'feature_57'): 0.109471,
    (26, 'feature_58'): 0.650378,
    (26, 'feature_59'): 1.195886,
    (26, 'feature_60'): -0.275775,
    (26, 'feature_61'): -1.275181,
    (26, 'feature_62'): -0.442438,
    (26, 'feature_63'): -0.313354,
    (26, 'feature_64'): -0.435501,
    (26, 'feature_65'): -1.413911,
    (26, 'feature_66'): 3.003890,
    (26, 'feature_67'): 0.458622,
    (26, 'feature_68'): 0.617764,
    (26, 'feature_69'): 1.438355,
    (26, 'feature_70'): 1.415438,
    (26, 'feature_71'): 0.944851,
    (26, 'feature_72'): 2.529821,
    (26, 'feature_73'): 0.103461,
    (26, 'feature_74'): 0.355609,
    (26, 'feature_75'): 3.437909,
    (26, 'feature_76'): 3.283568,
    (26, 'feature_77'): 0.886705,
    (26, 'feature_78'): 0.927187,
    (27, 'feature_00'): 3.936986,
    (27, 'feature_01'): -0.816895,
    (27, 'feature_02'): 3.692701,
    (27, 'feature_03'): 3.182293,
    (27, 'feature_04'): -0.341234,
    (27, 'feature_05'): 1.142155,
    (27, 'feature_06'): 0.347615,
    (27, 'feature_07'): 0.474967,
    (27, 'feature_08'): -1.436844,
    (27, 'feature_09'): 82.000000,
    (27, 'feature_10'): 3.000000,
    (27, 'feature_11'): 539.000000,
    (27, 'feature_12'): 0.848809,
    (27, 'feature_13'): 1.854786,
    (27, 'feature_14'): 0.899751,
    (27, 'feature_15'): -0.970623,
    (27, 'feature_16'): -0.711421,
    (27, 'feature_17'): -1.039402,
    (27, 'feature_18'): 0.550499,
    (27, 'feature_19'): 3.205250,
    (27, 'feature_20'): -0.129487,
    (27, 'feature_21'): -0.134027,
    (27, 'feature_22'): 1.063218,
    (27, 'feature_23'): 0.100825,
    (27, 'feature_24'): 2.983870,
    (27, 'feature_25'): 1.053391,
    (27, 'feature_26'): -0.996150,
    (27, 'feature_27'): -0.174806,
    (27, 'feature_28'): 0.531344,
    (27, 'feature_29'): -0.733654,
    (27, 'feature_30'): -0.639018,
    (27, 'feature_31'): -0.160936,
    (27, 'feature_32'): 2.791477,
    (27, 'feature_33'): -0.415989,
    (27, 'feature_34'): 3.287956,
    (27, 'feature_35'): 3.148027,
    (27, 'feature_36'): -0.866326,
    (27, 'feature_37'): -1.029145,
    (27, 'feature_38'): -1.330359,
    (27, 'feature_39'): 0.521619,
    (27, 'feature_40'): 0.972226,
    (27, 'feature_41'): 0.020280,
    (27, 'feature_42'): 1.207249,
    (27, 'feature_43'): -0.570806,
    (27, 'feature_44'): 0.055850,
    (27, 'feature_45'): 1.876266,
    (27, 'feature_46'): 1.150582,
    (27, 'feature_47'): 0.832912,
    (27, 'feature_48'): 0.561918,
    (27, 'feature_49'): 0.741669,
    (27, 'feature_50'): 0.404669,
    (27, 'feature_51'): 0.234003,
    (27, 'feature_52'): 1.073406,
    (27, 'feature_53'): 1.601939,
    (27, 'feature_54'): -0.155188,
    (27, 'feature_55'): 0.822823,
    (27, 'feature_56'): 1.738050,
    (27, 'feature_57'): 1.901297,
    (27, 'feature_58'): 1.317617,
    (27, 'feature_59'): 1.130453,
    (27, 'feature_60'): 1.299120,
    (27, 'feature_61'): -1.275181,
    (27, 'feature_62'): -0.394442,
    (27, 'feature_63'): -0.424745,
    (27, 'feature_64'): -0.578572,
    (27, 'feature_65'): 0.568769,
    (27, 'feature_66'): 4.163731,
    (27, 'feature_67'): 0.547478,
    (27, 'feature_68'): 1.015398,
    (27, 'feature_69'): 1.228739,
    (27, 'feature_70'): 1.081594,
    (27, 'feature_71'): 2.126414,
    (27, 'feature_72'): 1.205687,
    (27, 'feature_73'): -0.165440,
    (27, 'feature_74'): -0.050477,
    (27, 'feature_75'): 3.786745,
    (27, 'feature_76'): 3.223670,
    (27, 'feature_77'): 0.181590,
    (27, 'feature_78'): 0.327937,
    (28, 'feature_00'): 3.510596,
    (28, 'feature_01'): -0.538611,
    (28, 'feature_02'): 3.081784,
    (28, 'feature_03'): 3.553762,
    (28, 'feature_04'): -1.094796,
    (28, 'feature_05'): 1.121139,
    (28, 'feature_06'): 1.094312,
    (28, 'feature_07'): 0.640917,
    (28, 'feature_08'): -2.214025,
    (28, 'feature_09'): 9.000000,
    (28, 'feature_10'): 3.000000,
    (28, 'feature_11'): 25.000000,
    (28, 'feature_12'): 0.358280,
    (28, 'feature_13'): 0.201827,
    (28, 'feature_14'): 0.051352,
    (28, 'feature_15'): -0.635666,
    (28, 'feature_16'): -0.617204,
    (28, 'feature_17'): -0.743995,
    (28, 'feature_18'): -3.129544,
    (28, 'feature_19'): 3.233131,
    (28, 'feature_20'): -0.453187,
    (28, 'feature_21'): -0.268531,
    (28, 'feature_22'): -0.042179,
    (28, 'feature_23'): 0.090793,
    (28, 'feature_24'): 0.220330,
    (28, 'feature_25'): -0.831126,
    (28, 'feature_26'): -1.192038,
    (28, 'feature_27'): -0.091447,
    (28, 'feature_28'): 0.746151,
    (28, 'feature_29'): -0.713423,
    (28, 'feature_30'): -0.770919,
    (28, 'feature_31'): -0.155747,
    (28, 'feature_32'): 2.833474,
    (28, 'feature_33'): -0.571973,
    (28, 'feature_34'): 2.527267,
    (28, 'feature_35'): 2.862020,
    (28, 'feature_36'): -0.187681,
    (28, 'feature_37'): -1.711199,
    (28, 'feature_38'): -2.017045,
    (28, 'feature_39'): 0.840772,
    (28, 'feature_40'): -0.435708,
    (28, 'feature_41'): -0.048654,
    (28, 'feature_42'): 0.708126,
    (28, 'feature_43'): -0.290804,
    (28, 'feature_44'): 0.932571,
    (28, 'feature_45'): 0.954408,
    (28, 'feature_46'): 1.194647,
    (28, 'feature_47'): 0.600003,
    (28, 'feature_48'): 0.273342,
    (28, 'feature_49'): 0.269047,
    (28, 'feature_50'): 0.803827,
    (28, 'feature_51'): 0.590611,
    (28, 'feature_52'): 0.673798,
    (28, 'feature_53'): 1.606260,
    (28, 'feature_54'): 0.426359,
    (28, 'feature_55'): 0.500933,
    (28, 'feature_56'): 0.775849,
    (28, 'feature_57'): 1.750165,
    (28, 'feature_58'): 0.727474,
    (28, 'feature_59'): 0.603739,
    (28, 'feature_60'): 0.422402,
    (28, 'feature_61'): -1.275181,
    (28, 'feature_62'): -0.276580,
    (28, 'feature_63'): -0.334729,
    (28, 'feature_64'): -0.332914,
    (28, 'feature_65'): -1.677902,
    (28, 'feature_66'): 3.059176,
    (28, 'feature_67'): 0.298287,
    (28, 'feature_68'): -0.179875,
    (28, 'feature_69'): -0.067280,
    (28, 'feature_70'): 0.398488,
    (28, 'feature_71'): 0.662791,
    (28, 'feature_72'): 0.223852,
    (28, 'feature_73'): 0.045221,
    (28, 'feature_74'): 0.175570,
    (28, 'feature_75'): 0.705049,
    (28, 'feature_76'): 0.630108,
    (28, 'feature_77'): 0.024762,
    (28, 'feature_78'): 0.132983,
    (29, 'feature_00'): 3.958715,
    (29, 'feature_01'): -0.066671,
    (29, 'feature_02'): 3.129441,
    (29, 'feature_03'): 3.376085,
    (29, 'feature_04'): -1.212083,
    (29, 'feature_05'): 1.183166,
    (29, 'feature_06'): 0.883140,
    (29, 'feature_07'): 0.372877,
    (29, 'feature_08'): -1.258036,
    (29, 'feature_09'): 57.000000,
    (29, 'feature_10'): 1.000000,
    (29, 'feature_11'): 336.000000,
    (29, 'feature_12'): 0.295470,
    (29, 'feature_13'): 0.475068,
    (29, 'feature_14'): 0.353962,
    (29, 'feature_15'): -0.542430,
    (29, 'feature_16'): -0.251500,
    (29, 'feature_17'): -0.799353,
    (29, 'feature_18'): -0.227215,
    (29, 'feature_19'): 2.908640,
    (29, 'feature_20'): 0.443259,
    (29, 'feature_21'): -0.016385,
    (29, 'feature_22'): -0.411944,
    (29, 'feature_23'): -0.647582,
    (29, 'feature_24'): 2.033831,
    (29, 'feature_25'): 1.267834,
    (29, 'feature_26'): 0.362619,
    (29, 'feature_27'): -1.390535,
    (29, 'feature_28'): -1.880005,
    (29, 'feature_29'): -1.029640,
    (29, 'feature_30'): -0.871995,
    (29, 'feature_31'): -0.016294,
    (29, 'feature_32'): 2.771026,
    (29, 'feature_33'): 0.061867,
    (29, 'feature_34'): 3.099031,
    (29, 'feature_35'): 3.244698,
    (29, 'feature_36'): -0.927842,
    (29, 'feature_37'): -1.415889,
    (29, 'feature_38'): -1.760117,
    (29, 'feature_39'): 1.009808,
    (29, 'feature_40'): 0.307549,
    (29, 'feature_41'): 1.634030,
    (29, 'feature_42'): 0.455093,
    (29, 'feature_43'): -0.396475,
    (29, 'feature_44'): -0.647948,
    (29, 'feature_45'): 2.610356,
    (29, 'feature_46'): 1.145501,
    (29, 'feature_47'): 0.736506,
    (29, 'feature_48'): -0.003415,
    (29, 'feature_49'): 0.615775,
    (29, 'feature_50'): 0.503252,
    (29, 'feature_51'): -0.587757,
    (29, 'feature_52'): 1.183675,
    (29, 'feature_53'): -0.171550,
    (29, 'feature_54'): -0.635554,
    (29, 'feature_55'): -0.147109,
    (29, 'feature_56'): 1.933254,
    (29, 'feature_57'): 0.419499,
    (29, 'feature_58'): 0.440748,
    (29, 'feature_59'): -0.253923,
    (29, 'feature_60'): 0.470296,
    (29, 'feature_61'): -1.275181,
    (29, 'feature_62'): -0.341182,
    (29, 'feature_63'): -0.547681,
    (29, 'feature_64'): -0.375505,
    (29, 'feature_65'): -0.587784,
    (29, 'feature_66'): 1.891227,
    (29, 'feature_67'): 0.078034,
    (29, 'feature_68'): 0.600033,
    (29, 'feature_69'): 0.186102,
    (29, 'feature_70'): 0.411432,
    (29, 'feature_71'): 0.607202,
    (29, 'feature_72'): 0.561938,
    (29, 'feature_73'): 0.916576,
    (29, 'feature_74'): 1.129362,
    (29, 'feature_75'): 6.951842,
    (29, 'feature_76'): 10.016251,
    (29, 'feature_77'): 2.368672,
    (29, 'feature_78'): 2.314198,
    (30, 'feature_00'): 3.173539,
    (30, 'feature_01'): -0.195566,
    (30, 'feature_02'): 3.548015,
    (30, 'feature_03'): 3.097099,
    (30, 'feature_04'): -0.736404,
    (30, 'feature_05'): 1.119664,
    (30, 'feature_06'): 0.853305,
    (30, 'feature_07'): 0.426892,
    (30, 'feature_08'): -1.504480,
    (30, 'feature_09'): 81.000000,
    (30, 'feature_10'): 2.000000,
    (30, 'feature_11'): 534.000000,
    (30, 'feature_12'): 0.623820,
    (30, 'feature_13'): 0.784778,
    (30, 'feature_14'): 1.246606,
    (30, 'feature_15'): -0.711657,
    (30, 'feature_16'): -0.350926,
    (30, 'feature_17'): -0.766431,
    (30, 'feature_18'): 0.277664,
    (30, 'feature_19'): 2.250896,
    (30, 'feature_20'): 0.251397,
    (30, 'feature_21'): 0.109864,
    (30, 'feature_22'): 0.149313,
    (30, 'feature_23'): -0.050437,
    (30, 'feature_24'): 1.755153,
    (30, 'feature_25'): 1.136264,
    (30, 'feature_26'): -1.136783,
    (30, 'feature_27'): -0.985110,
    (30, 'feature_28'): -0.427006,
    (30, 'feature_29'): -0.673519,
    (30, 'feature_30'): -0.623362,
    (30, 'feature_31'): 0.096575,
    (30, 'feature_32'): 2.603460,
    (30, 'feature_33'): -0.654686,
    (30, 'feature_34'): 2.789652,
    (30, 'feature_35'): 2.826175,
    (30, 'feature_36'): -0.815835,
    (30, 'feature_37'): -0.025968,
    (30, 'feature_38'): -0.090140,
    (30, 'feature_39'): 0.315758,
    (30, 'feature_40'): -0.501679,
    (30, 'feature_41'): 0.754713,
    (30, 'feature_42'): -0.229859,
    (30, 'feature_43'): -0.305364,
    (30, 'feature_44'): -0.511550,
    (30, 'feature_45'): 0.152285,
    (30, 'feature_46'): -0.841227,
    (30, 'feature_47'): -0.065723,
    (30, 'feature_48'): -0.122480,
    (30, 'feature_49'): 0.031396,
    (30, 'feature_50'): 0.252049,
    (30, 'feature_51'): 0.099182,
    (30, 'feature_52'): 1.242389,
    (30, 'feature_53'): -0.339274,
    (30, 'feature_54'): -0.320001,
    (30, 'feature_55'): -0.152217,
    (30, 'feature_56'): -0.131302,
    (30, 'feature_57'): -0.195076,
    (30, 'feature_58'): -0.007445,
    (30, 'feature_59'): -0.560651,
    (30, 'feature_60'): 0.057026,
    (30, 'feature_61'): -1.275181,
    (30, 'feature_62'): -0.453196,
    (30, 'feature_63'): -0.373061,
    (30, 'feature_64'): -0.404986,
    (30, 'feature_65'): -0.615736,
    (30, 'feature_66'): 2.335820,
    (30, 'feature_67'): 0.824626,
    (30, 'feature_68'): 0.707431,
    (30, 'feature_69'): 1.011204,
    (30, 'feature_70'): 0.917808,
    (30, 'feature_71'): 0.435990,
    (30, 'feature_72'): 0.904955,
    (30, 'feature_73'): -0.254245,
    (30, 'feature_74'): -0.141560,
    (30, 'feature_75'): 0.544881,
    (30, 'feature_76'): 0.732551,
    (30, 'feature_77'): 0.029713,
    (30, 'feature_78'): 0.028917,
    (31, 'feature_00'): 3.163152,
    (31, 'feature_01'): -0.478260,
    (31, 'feature_02'): 3.288042,
    (31, 'feature_03'): 3.529703,
    (31, 'feature_04'): -0.781037,
    (31, 'feature_05'): 1.230066,
    (31, 'feature_06'): 0.787423,
    (31, 'feature_07'): 0.453916,
    (31, 'feature_08'): -1.103508,
    (31, 'feature_09'): 44.000000,
    (31, 'feature_10'): 3.000000,
    (31, 'feature_11'): 24.000000,
    (31, 'feature_12'): -0.130307,
    (31, 'feature_13'): 0.068522,
    (31, 'feature_14'): 0.084979,
    (31, 'feature_15'): -0.711560,
    (31, 'feature_16'): -0.798192,
    (31, 'feature_17'): -0.635255,
    (31, 'feature_18'): -0.698825,
    (31, 'feature_19'): 3.055189,
    (31, 'feature_20'): 0.160455,
    (31, 'feature_21'): -0.107653,
    (31, 'feature_22'): -0.815329,
    (31, 'feature_23'): -0.794446,
    (31, 'feature_24'): 1.347932,
    (31, 'feature_25'): 0.877272,
    (31, 'feature_26'): 0.113094,
    (31, 'feature_27'): -1.801203,
    (31, 'feature_28'): -2.088478,
    (31, 'feature_29'): -0.868496,
    (31, 'feature_30'): -0.692478,
    (31, 'feature_31'): -0.175931,
    (31, 'feature_32'): 2.899077,
    (31, 'feature_33'): -0.201802,
    (31, 'feature_34'): 3.319461,
    (31, 'feature_35'): 3.012369,
    (31, 'feature_36'): -0.406770,
    (31, 'feature_37'): -1.035051,
    (31, 'feature_38'): -1.059037,
    (31, 'feature_39'): 1.256223,
    (31, 'feature_40'): 0.346150,
    (31, 'feature_41'): 0.939708,
    (31, 'feature_42'): 0.616872,
    (31, 'feature_43'): -0.053842,
    (31, 'feature_44'): 0.040407,
    (31, 'feature_45'): 1.045824,
    (31, 'feature_46'): 1.290680,
    (31, 'feature_47'): 1.001266,
    (31, 'feature_48'): 0.059014,
    (31, 'feature_49'): 0.902714,
    (31, 'feature_50'): 1.610027,
    (31, 'feature_51'): -0.077590,
    (31, 'feature_52'): 1.062141,
    (31, 'feature_53'): 1.584386,
    (31, 'feature_54'): 0.683545,
    (31, 'feature_55'): 0.823477,
    (31, 'feature_56'): 1.086005,
    (31, 'feature_57'): 1.577737,
    (31, 'feature_58'): 1.921278,
    (31, 'feature_59'): 0.098853,
    (31, 'feature_60'): 1.097613,
    (31, 'feature_61'): -1.275181,
    (31, 'feature_62'): -0.403121,
    (31, 'feature_63'): -0.261756,
    (31, 'feature_64'): -0.280641,
    (31, 'feature_65'): -0.644154,
    (31, 'feature_66'): 3.345170,
    (31, 'feature_67'): -0.277954,
    (31, 'feature_68'): 0.161548,
    (31, 'feature_69'): 0.090727,
    (31, 'feature_70'): 0.055101,
    (31, 'feature_71'): -0.029785,
    (31, 'feature_72'): 0.091748,
    (31, 'feature_73'): 1.267962,
    (31, 'feature_74'): 1.254009,
    (31, 'feature_75'): 0.139119,
    (31, 'feature_76'): 0.262129,
    (31, 'feature_77'): 1.307964,
    (31, 'feature_78'): 1.669839,
    (32, 'feature_00'): 3.368008,
    (32, 'feature_01'): -1.238870,
    (32, 'feature_02'): 3.528969,
    (32, 'feature_03'): 3.240196,
    (32, 'feature_04'): -0.410036,
    (32, 'feature_05'): 0.963363,
    (32, 'feature_06'): 0.684894,
    (32, 'feature_07'): 0.475044,
    (32, 'feature_08'): -1.652292,
    (32, 'feature_09'): 26.000000,
    (32, 'feature_10'): 12.000000,
    (32, 'feature_11'): 158.000000,
    (32, 'feature_12'): -0.233784,
    (32, 'feature_13'): 0.278864,
    (32, 'feature_14'): 0.090098,
    (32, 'feature_15'): -0.598102,
    (32, 'feature_16'): -0.502853,
    (32, 'feature_17'): -0.712108,
    (32, 'feature_18'): -0.779284,
    (32, 'feature_19'): 1.958240,
    (32, 'feature_20'): 0.811378,
    (32, 'feature_21'): 0.062979,
    (32, 'feature_22'): -0.062264,
    (32, 'feature_23'): 0.328656,
    (32, 'feature_24'): 1.973440,
    (32, 'feature_25'): 1.499756,
    (32, 'feature_26'): 2.000274,
    (32, 'feature_27'): -1.505330,
    (32, 'feature_28'): -1.604580,
    (32, 'feature_29'): -0.319669,
    (32, 'feature_30'): -0.852470,
    (32, 'feature_31'): 0.085925,
    (32, 'feature_32'): 2.672311,
    (32, 'feature_33'): 0.405482,
    (32, 'feature_34'): 3.551963,
    (32, 'feature_35'): 3.278015,
    (32, 'feature_36'): -0.015373,
    (32, 'feature_37'): -1.008539,
    (32, 'feature_38'): -1.825385,
    (32, 'feature_39'): -1.004468,
    (32, 'feature_40'): 0.338946,
    (32, 'feature_41'): -0.217616,
    (32, 'feature_42'): -1.247960,
    (32, 'feature_43'): -0.542911,
    (32, 'feature_44'): -0.430872,
    (32, 'feature_45'): 1.227600,
    (32, 'feature_46'): 0.043799,
    (32, 'feature_47'): -0.321336,
    (32, 'feature_48'): 0.170428,
    (32, 'feature_49'): -0.674538,
    (32, 'feature_50'): -0.141119,
    (32, 'feature_51'): 0.208001,
    (32, 'feature_52'): 0.916416,
    (32, 'feature_53'): -0.541129,
    (32, 'feature_54'): 0.309621,
    (32, 'feature_55'): -0.170980,
    (32, 'feature_56'): 1.431467,
    (32, 'feature_57'): 0.598191,
    (32, 'feature_58'): -0.416403,
    (32, 'feature_59'): 0.686077,
    (32, 'feature_60'): -0.654318,
    (32, 'feature_61'): -1.275181,
    (32, 'feature_62'): -0.339270,
    (32, 'feature_63'): -0.279586,
    (32, 'feature_64'): -0.347068,
    (32, 'feature_65'): -0.961778,
    (32, 'feature_66'): 2.976419,
    (32, 'feature_67'): -0.190133,
    (32, 'feature_68'): 0.275995,
    (32, 'feature_69'): 0.090056,
    (32, 'feature_70'): -0.159033,
    (32, 'feature_71'): 0.186851,
    (32, 'feature_72'): 0.066140,
    (32, 'feature_73'): -0.248829,
    (32, 'feature_74'): -0.030216,
    (32, 'feature_75'): -0.306492,
    (32, 'feature_76'): -0.100647,
    (32, 'feature_77'): -0.313567,
    (32, 'feature_78'): -0.100272,
    (33, 'feature_00'): 3.131203,
    (33, 'feature_01'): -0.659437,
    (33, 'feature_02'): 3.156818,
    (33, 'feature_03'): 3.522580,
    (33, 'feature_04'): -0.682810,
    (33, 'feature_05'): 1.342655,
    (33, 'feature_06'): 0.665984,
    (33, 'feature_07'): 0.383806,
    (33, 'feature_08'): -1.885080,
    (33, 'feature_09'): 11.000000,
    (33, 'feature_10'): 7.000000,
    (33, 'feature_11'): 76.000000,
    (33, 'feature_12'): 0.646753,
    (33, 'feature_13'): 1.256358,
    (33, 'feature_14'): 0.739470,
    (33, 'feature_15'): -0.574699,
    (33, 'feature_16'): -0.404904,
    (33, 'feature_17'): -0.541519,
    (33, 'feature_18'): 0.168975,
    (33, 'feature_19'): 1.999272,
    (33, 'feature_20'): 0.320679,
    (33, 'feature_21'): -0.352143,
    (33, 'feature_22'): -0.332837,
    (33, 'feature_23'): -0.163710,
    (33, 'feature_24'): 0.676220,
    (33, 'feature_25'): -0.126100,
    (33, 'feature_26'): -0.199542,
    (33, 'feature_27'): -0.115075,
    (33, 'feature_28'): -0.153173,
    (33, 'feature_29'): -0.790685,
    (33, 'feature_30'): -0.535115,
    (33, 'feature_31'): -0.298453,
    (33, 'feature_32'): 2.543671,
    (33, 'feature_33'): 0.215852,
    (33, 'feature_34'): 3.322385,
    (33, 'feature_35'): 3.067891,
    (33, 'feature_36'): 0.349683,
    (33, 'feature_37'): 0.087815,
    (33, 'feature_38'): -0.228722,
    (33, 'feature_39'): 1.378871,
    (33, 'feature_40'): 0.736902,
    (33, 'feature_41'): 0.485837,
    (33, 'feature_42'): 0.373742,
    (33, 'feature_43'): -0.361288,
    (33, 'feature_44'): 0.523084,
    (33, 'feature_45'): 0.303391,
    (33, 'feature_46'): 0.134675,
    (33, 'feature_47'): 0.542742,
    (33, 'feature_48'): -0.350915,
    (33, 'feature_49'): 0.591056,
    (33, 'feature_50'): 0.873527,
    (33, 'feature_51'): 0.943739,
    (33, 'feature_52'): 0.734061,
    (33, 'feature_53'): 0.915036,
    (33, 'feature_54'): -0.627159,
    (33, 'feature_55'): -0.219154,
    (33, 'feature_56'): 0.580117,
    (33, 'feature_57'): 0.420290,
    (33, 'feature_58'): 0.700121,
    (33, 'feature_59'): 0.081578,
    (33, 'feature_60'): 0.572185,
    (33, 'feature_61'): -1.275181,
    (33, 'feature_62'): -0.337379,
    (33, 'feature_63'): -0.269830,
    (33, 'feature_64'): -0.276134,
    (33, 'feature_65'): 0.202721,
    (33, 'feature_66'): 1.885601,
    (33, 'feature_67'): 0.341088,
    (33, 'feature_68'): 0.254203,
    (33, 'feature_69'): 0.110008,
    (33, 'feature_70'): 0.717665,
    (33, 'feature_71'): 1.017407,
    (33, 'feature_72'): 0.778950,
    (33, 'feature_73'): -0.060347,
    (33, 'feature_74'): -0.030762,
    (33, 'feature_75'): 0.843712,
    (33, 'feature_76'): 0.653514,
    (33, 'feature_77'): 0.172209,
    (33, 'feature_78'): 0.147376,
    (34, 'feature_00'): 3.435512,
    (34, 'feature_01'): 0.097569,
    (34, 'feature_02'): 3.203450,
    (34, 'feature_03'): 3.235055,
    (34, 'feature_04'): -0.595968,
    (34, 'feature_05'): 1.679520,
    (34, 'feature_06'): 1.596661,
    (34, 'feature_07'): 0.620067,
    (34, 'feature_08'): -2.346262,
    (34, 'feature_09'): 42.000000,
    (34, 'feature_10'): 5.000000,
    (34, 'feature_11'): 150.000000,
    (34, 'feature_12'): 0.851463,
    (34, 'feature_13'): 0.784189,
    (34, 'feature_14'): 2.511799,
    (34, 'feature_15'): -0.799417,
    (34, 'feature_16'): -0.494508,
    (34, 'feature_17'): -0.462186,
    (34, 'feature_18'): -2.974867,
    (34, 'feature_19'): 3.209658,
    (34, 'feature_20'): -0.103817,
    (34, 'feature_21'): -0.275483,
    (34, 'feature_22'): 0.733441,
    (34, 'feature_23'): 1.047878,
    (34, 'feature_24'): -0.308685,
    (34, 'feature_25'): -0.778517,
    (34, 'feature_26'): -0.154889,
    (34, 'feature_27'): 1.556985,
    (34, 'feature_28'): 2.271212,
    (34, 'feature_29'): -0.552274,
    (34, 'feature_30'): -0.414992,
    (34, 'feature_31'): -0.284826,
    (34, 'feature_32'): 2.922466,
    (34, 'feature_33'): 0.153479,
    (34, 'feature_34'): 2.811527,
    (34, 'feature_35'): 2.921942,
    (34, 'feature_36'): -0.467182,
    (34, 'feature_37'): -1.863372,
    (34, 'feature_38'): -2.188352,
    (34, 'feature_39'): 0.936006,
    (34, 'feature_40'): 0.530295,
    (34, 'feature_41'): 0.946823,
    (34, 'feature_42'): -0.809370,
    (34, 'feature_43'): 0.036882,
    (34, 'feature_44'): -0.358867,
    (34, 'feature_45'): 1.580429,
    (34, 'feature_46'): 0.226126,
    (34, 'feature_47'): 0.302171,
    (34, 'feature_48'): 0.555784,
    (34, 'feature_49'): -0.413440,
    (34, 'feature_50'): 0.954343,
    (34, 'feature_51'): -0.165923,
    (34, 'feature_52'): 2.176920,
    (34, 'feature_53'): -0.979955,
    (34, 'feature_54'): 0.134294,
    (34, 'feature_55'): -0.607851,
    (34, 'feature_56'): 1.681813,
    (34, 'feature_57'): 0.378224,
    (34, 'feature_58'): 0.494567,
    (34, 'feature_59'): 0.608613,
    (34, 'feature_60'): -0.525847,
    (34, 'feature_61'): -1.275181,
    (34, 'feature_62'): -0.433487,
    (34, 'feature_63'): -0.261390,
    (34, 'feature_64'): -0.096022,
    (34, 'feature_65'): -1.480620,
    (34, 'feature_66'): 3.214448,
    (34, 'feature_67'): 0.956792,
    (34, 'feature_68'): 0.898538,
    (34, 'feature_69'): 2.221581,
    (34, 'feature_70'): 0.694664,
    (34, 'feature_71'): 0.088554,
    (34, 'feature_72'): 1.442000,
    (34, 'feature_73'): -0.278385,
    (34, 'feature_74'): -0.103714,
    (34, 'feature_75'): 0.091430,
    (34, 'feature_76'): 0.236411,
    (34, 'feature_77'): 0.007355,
    (34, 'feature_78'): 0.165148,
    (35, 'feature_00'): 3.435329,
    (35, 'feature_01'): -0.753699,
    (35, 'feature_02'): 3.014601,
    (35, 'feature_03'): 3.734002,
    (35, 'feature_04'): -0.972336,
    (35, 'feature_05'): 1.099953,
    (35, 'feature_06'): 1.003350,
    (35, 'feature_07'): 0.427752,
    (35, 'feature_08'): -1.787689,
    (35, 'feature_09'): 25.000000,
    (35, 'feature_10'): 7.000000,
    (35, 'feature_11'): 195.000000,
    (35, 'feature_12'): 0.478906,
    (35, 'feature_13'): 0.403514,
    (35, 'feature_14'): 0.698781,
    (35, 'feature_15'): -0.329375,
    (35, 'feature_16'): -0.388264,
    (35, 'feature_17'): -0.314414,
    (35, 'feature_18'): 3.005118,
    (35, 'feature_19'): -0.575803,
    (35, 'feature_20'): 0.147754,
    (35, 'feature_21'): -0.155405,
    (35, 'feature_22'): -1.052496,
    (35, 'feature_23'): -0.002176,
    (35, 'feature_24'): 1.650035,
    (35, 'feature_25'): 0.913920,
    (35, 'feature_26'): 0.111702,
    (35, 'feature_27'): -1.563342,
    (35, 'feature_28'): -1.462345,
    (35, 'feature_29'): -0.159507,
    (35, 'feature_30'): -0.558569,
    (35, 'feature_31'): -0.143225,
    (35, 'feature_32'): 3.153672,
    (35, 'feature_33'): 1.165126,
    (35, 'feature_34'): 3.121894,
    (35, 'feature_35'): 2.888050,
    (35, 'feature_36'): 2.128495,
    (35, 'feature_37'): 0.497890,
    (35, 'feature_38'): 0.096050,
    (35, 'feature_39'): 1.010427,
    (35, 'feature_40'): -1.222002,
    (35, 'feature_41'): 2.059643,
    (35, 'feature_42'): -0.991660,
    (35, 'feature_43'): -1.319574,
    (35, 'feature_44'): -0.204090,
    (35, 'feature_45'): -0.098942,
    (35, 'feature_46'): -1.356683,
    (35, 'feature_47'): -0.265023,
    (35, 'feature_48'): -0.225226,
    (35, 'feature_49'): 0.347769,
    (35, 'feature_50'): 0.960363,
    (35, 'feature_51'): -1.187350,
    (35, 'feature_52'): 2.237002,
    (35, 'feature_53'): -1.655461,
    (35, 'feature_54'): -0.881900,
    (35, 'feature_55'): -0.116357,
    (35, 'feature_56'): -0.326911,
    (35, 'feature_57'): -0.811340,
    (35, 'feature_58'): -0.778488,
    (35, 'feature_59'): -0.595511,
    (35, 'feature_60'): 0.082265,
    (35, 'feature_61'): -1.275181,
    (35, 'feature_62'): -0.333411,
    (35, 'feature_63'): -0.262894,
    (35, 'feature_64'): -0.327824,
    (35, 'feature_65'): 3.285433,
    (35, 'feature_66'): -0.427485,
    (35, 'feature_67'): 0.306444,
    (35, 'feature_68'): 0.524980,
    (35, 'feature_69'): 0.743504,
    (35, 'feature_70'): 0.142597,
    (35, 'feature_71'): 0.286472,
    (35, 'feature_72'): 0.376100,
    (35, 'feature_73'): -0.102826,
    (35, 'feature_74'): -0.097081,
    (35, 'feature_75'): -0.115849,
    (35, 'feature_76'): -0.135580,
    (35, 'feature_77'): -0.015578,
    (35, 'feature_78'): -0.028191,
    (36, 'feature_00'): 3.185427,
    (36, 'feature_01'): -0.377582,
    (36, 'feature_02'): 3.090469,
    (36, 'feature_03'): 3.171644,
    (36, 'feature_04'): -0.476881,
    (36, 'feature_05'): 1.449648,
    (36, 'feature_06'): 0.733523,
    (36, 'feature_07'): 0.524119,
    (36, 'feature_08'): -1.044434,
    (36, 'feature_09'): 49.000000,
    (36, 'feature_10'): 7.000000,
    (36, 'feature_11'): 297.000000,
    (36, 'feature_12'): 1.364073,
    (36, 'feature_13'): 0.540443,
    (36, 'feature_14'): 0.909310,
    (36, 'feature_15'): -0.365137,
    (36, 'feature_16'): -0.371249,
    (36, 'feature_17'): -0.541907,
    (36, 'feature_18'): 1.949706,
    (36, 'feature_19'): -1.109278,
    (36, 'feature_20'): -0.020168,
    (36, 'feature_21'): 0.089721,
    (36, 'feature_22'): -0.870483,
    (36, 'feature_23'): -0.254126,
    (36, 'feature_24'): 1.084532,
    (36, 'feature_25'): 0.471281,
    (36, 'feature_26'): -0.367174,
    (36, 'feature_27'): -1.014297,
    (36, 'feature_28'): -1.991747,
    (36, 'feature_29'): -0.432912,
    (36, 'feature_30'): -0.583480,
    (36, 'feature_31'): 0.079240,
    (36, 'feature_32'): 2.987374,
    (36, 'feature_33'): -0.702395,
    (36, 'feature_34'): 3.306531,
    (36, 'feature_35'): 3.185636,
    (36, 'feature_36'): -0.775737,
    (36, 'feature_37'): 0.321189,
    (36, 'feature_38'): -0.017911,
    (36, 'feature_39'): 0.913403,
    (36, 'feature_40'): 0.104324,
    (36, 'feature_41'): -0.431190,
    (36, 'feature_42'): -0.610862,
    (36, 'feature_43'): -0.703052,
    (36, 'feature_44'): -0.291703,
    (36, 'feature_45'): 0.456106,
    (36, 'feature_46'): -0.336485,
    (36, 'feature_47'): 0.195010,
    (36, 'feature_48'): -0.635903,
    (36, 'feature_49'): -0.084846,
    (36, 'feature_50'): 0.840975,
    (36, 'feature_51'): 0.285581,
    (36, 'feature_52'): 0.090543,
    (36, 'feature_53'): -0.101928,
    (36, 'feature_54'): -0.345839,
    (36, 'feature_55'): 0.050004,
    (36, 'feature_56'): 0.297977,
    (36, 'feature_57'): -0.718631,
    (36, 'feature_58'): 0.471840,
    (36, 'feature_59'): -0.213817,
    (36, 'feature_60'): 0.087194,
    (36, 'feature_61'): -1.275181,
    (36, 'feature_62'): -0.291706,
    (36, 'feature_63'): -0.106148,
    (36, 'feature_64'): -0.404137,
    (36, 'feature_65'): 2.057968,
    (36, 'feature_66'): -1.421511,
    (36, 'feature_67'): 1.028342,
    (36, 'feature_68'): 0.528886,
    (36, 'feature_69'): 0.413159,
    (36, 'feature_70'): 1.831115,
    (36, 'feature_71'): 0.077209,
    (36, 'feature_72'): 1.266410,
    (36, 'feature_73'): -0.105521,
    (36, 'feature_74'): -0.103522,
    (36, 'feature_75'): 0.861319,
    (36, 'feature_76'): 1.071085,
    (36, 'feature_77'): 0.055652,
    (36, 'feature_78'): 0.051880,
    (37, 'feature_00'): 3.556767,
    (37, 'feature_01'): -0.610766,
    (37, 'feature_02'): 3.638054,
    (37, 'feature_03'): 3.396827,
    (37, 'feature_04'): -0.865197,
    (37, 'feature_05'): 1.441855,
    (37, 'feature_06'): 1.138223,
    (37, 'feature_07'): 0.508513,
    (37, 'feature_08'): -1.765435,
    (37, 'feature_09'): 34.000000,
    (37, 'feature_10'): 4.000000,
    (37, 'feature_11'): 214.000000,
    (37, 'feature_12'): 0.260464,
    (37, 'feature_13'): -0.005956,
    (37, 'feature_14'): -0.060403,
    (37, 'feature_15'): -0.558848,
    (37, 'feature_16'): -0.530056,
    (37, 'feature_17'): -0.643644,
    (37, 'feature_18'): -1.557064,
    (37, 'feature_19'): 2.247708,
    (37, 'feature_20'): -0.824507,
    (37, 'feature_21'): -0.241621,
    (37, 'feature_22'): -0.285240,
    (37, 'feature_23'): -0.229761,
    (37, 'feature_24'): 0.070222,
    (37, 'feature_25'): -0.676569,
    (37, 'feature_26'): 0.509950,
    (37, 'feature_27'): -0.418716,
    (37, 'feature_28'): -1.107131,
    (37, 'feature_29'): -0.578253,
    (37, 'feature_30'): -0.987387,
    (37, 'feature_31'): -0.316847,
    (37, 'feature_32'): 2.602535,
    (37, 'feature_33'): 1.162431,
    (37, 'feature_34'): 2.760442,
    (37, 'feature_35'): 2.968204,
    (37, 'feature_36'): -1.274045,
    (37, 'feature_37'): -1.081049,
    (37, 'feature_38'): -1.063444,
    (37, 'feature_39'): 0.472845,
    (37, 'feature_40'): -1.324151,
    (37, 'feature_41'): -0.449838,
    (37, 'feature_42'): -0.122470,
    (37, 'feature_43'): -0.461104,
    (37, 'feature_44'): -1.116860,
    (37, 'feature_45'): 0.755360,
    (37, 'feature_46'): -0.484826,
    (37, 'feature_47'): -0.147106,
    (37, 'feature_48'): -0.283695,
    (37, 'feature_49'): -0.287267,
    (37, 'feature_50'): 0.046282,
    (37, 'feature_51'): -0.099096,
    (37, 'feature_52'): -0.339083,
    (37, 'feature_53'): -0.483794,
    (37, 'feature_54'): 0.220357,
    (37, 'feature_55'): -1.008615,
    (37, 'feature_56'): 1.219831,
    (37, 'feature_57'): -0.374687,
    (37, 'feature_58'): -0.079300,
    (37, 'feature_59'): -0.198310,
    (37, 'feature_60'): -0.089621,
    (37, 'feature_61'): -1.275181,
    (37, 'feature_62'): -0.276069,
    (37, 'feature_63'): -0.367680,
    (37, 'feature_64'): -0.327152,
    (37, 'feature_65'): -0.394588,
    (37, 'feature_66'): 1.937018,
    (37, 'feature_67'): 0.114911,
    (37, 'feature_68'): 0.094971,
    (37, 'feature_69'): -0.122071,
    (37, 'feature_70'): 0.372170,
    (37, 'feature_71'): -0.124795,
    (37, 'feature_72'): 0.006410,
    (37, 'feature_73'): 0.133439,
    (37, 'feature_74'): 0.288192,
    (37, 'feature_75'): 0.586589,
    (37, 'feature_76'): 0.606713,
    (37, 'feature_77'): 0.203193,
    (37, 'feature_78'): 0.371953,
    (38, 'feature_00'): 3.378834,
    (38, 'feature_01'): -0.213640,
    (38, 'feature_02'): 3.037545,
    (38, 'feature_03'): 3.319021,
    (38, 'feature_04'): -0.788946,
    (38, 'feature_05'): 1.540720,
    (38, 'feature_06'): 1.023925,
    (38, 'feature_07'): 0.556460,
    (38, 'feature_08'): -1.713150,
    (38, 'feature_09'): 50.000000,
    (38, 'feature_10'): 1.000000,
    (38, 'feature_11'): 522.000000,
    (38, 'feature_12'): -0.111710,
    (38, 'feature_13'): 0.075662,
    (38, 'feature_14'): -0.121446,
    (38, 'feature_15'): -1.210433,
    (38, 'feature_16'): -0.781683,
    (38, 'feature_17'): -0.908807,
    (38, 'feature_18'): -3.628248,
    (38, 'feature_19'): 2.169930,
    (38, 'feature_20'): -0.407392,
    (38, 'feature_21'): -0.092133,
    (38, 'feature_22'): 1.349818,
    (38, 'feature_23'): 0.867710,
    (38, 'feature_24'): 0.622873,
    (38, 'feature_25'): -0.160515,
    (38, 'feature_26'): 0.582277,
    (38, 'feature_27'): 0.427575,
    (38, 'feature_28'): 0.121049,
    (38, 'feature_29'): -0.545744,
    (38, 'feature_30'): -1.016142,
    (38, 'feature_31'): -0.135769,
    (38, 'feature_32'): 2.922949,
    (38, 'feature_33'): -0.794040,
    (38, 'feature_34'): 2.986512,
    (38, 'feature_35'): 3.157950,
    (38, 'feature_36'): -0.618797,
    (38, 'feature_37'): -1.468656,
    (38, 'feature_38'): -1.448584,
    (38, 'feature_39'): -0.360112,
    (38, 'feature_40'): 0.244883,
    (38, 'feature_41'): -0.062132,
    (38, 'feature_42'): 0.034904,
    (38, 'feature_43'): -0.056696,
    (38, 'feature_44'): -0.146467,
    (38, 'feature_45'): 0.762359,
    (38, 'feature_46'): 0.666838,
    (38, 'feature_47'): 0.139382,
    (38, 'feature_48'): 0.222590,
    (38, 'feature_49'): 0.052657,
    (38, 'feature_50'): -0.387095,
    (38, 'feature_51'): 0.475139,
    (38, 'feature_52'): 0.411692,
    (38, 'feature_53'): 0.275864,
    (38, 'feature_54'): 0.754020,
    (38, 'feature_55'): 0.647276,
    (38, 'feature_56'): 1.218158,
    (38, 'feature_57'): 0.263234,
    (38, 'feature_58'): 0.428090,
    (38, 'feature_59'): 0.429107,
    (38, 'feature_60'): 0.218974,
    (38, 'feature_61'): -1.275181,
    (38, 'feature_62'): -0.473674,
    (38, 'feature_63'): -0.397880,
    (38, 'feature_64'): -0.314145,
    (38, 'feature_65'): -1.943346,
    (38, 'feature_66'): 2.334590,
    (38, 'feature_67'): -0.179209,
    (38, 'feature_68'): -0.125444,
    (38, 'feature_69'): -0.142959,
    (38, 'feature_70'): 0.086960,
    (38, 'feature_71'): 0.386853,
    (38, 'feature_72'): -0.064715,
    (38, 'feature_73'): -0.336321,
    (38, 'feature_74'): -0.111983,
    (38, 'feature_75'): -0.029611,
    (38, 'feature_76'): 0.101167,
    (38, 'feature_77'): 0.097256,
    (38, 'feature_78'): 0.157802,
}


#### Global constants for feature processing

In [276]:
FEATURE_COLS = [f'feature_{i:02d}' for i in range(79)]
FEATURE_NULL_COLS = [f'{col}_is_null' for col in FEATURE_COLS]
MODEL_COLS = ['symbol_id', 'weight'] + FEATURE_COLS + FEATURE_NULL_COLS
FEATURE_TYPES = {
    # Int16 features
    'feature_11': pl.Int16,
    
    # Int8 features
    'feature_09': pl.Int8,
    'feature_10': pl.Int8,
    
    # All other features are Float32
    **{f'feature_{i:02d}': pl.Float32 for i in range(79) 
       if i not in [9, 10, 11]}
}

In [277]:

def preprocess_features(df: pl.DataFrame) -> np.ndarray:
    """Preprocess features and return numpy array ready for prediction"""
    global last_known_values
    
    # Extract all needed data as numpy arrays once
    symbol_ids = df['symbol_id'].to_numpy()
    weights = df['weight'].to_numpy()
    
    # Preallocate final array (features + null flags + weight + symbol_id)
    n_rows = len(df)
    processed = np.zeros((n_rows, len(MODEL_COLS)), dtype=np.float32)
    
    # Fill symbol_id and weight columns first
    processed[:, 0] = symbol_ids
    processed[:, 1] = weights
    
    # Process features and create null flags
    for i, col in enumerate(FEATURE_COLS):
        values = df[col].to_numpy()
        null_mask = np.isnan(values)
        
        # Fill null values
        if null_mask.any():
            for idx in np.where(null_mask)[0]:
                sym = symbol_ids[idx]
                values[idx] = last_known_values.get((sym, col)) # TODO: No loner accounting for unseen symbols, need to fix.
        
        # Update last known values
        for sym, val in zip(symbol_ids[~null_mask], values[~null_mask]):
            last_known_values[(sym, col)] = val
            
        # Set feature values and null flags
        processed[:, 2+i] = values  # Features come after symbol_id and weight
        processed[:, 2+len(FEATURE_COLS)+i] = null_mask  # Null flags at the end
    
    return processed

### Load model

In [278]:
import lightgbm

In [279]:
# My models
# "val_r2_0.0122524.joblib"
# "backup_best_valr2.joblib"
# model_iter_1000_valr2_0.013022717281426566.joblib
# model_iter_221_valr2_0.006522601408196826.joblib

# Benchmark Models
# LGBMV1_1.joblib

model_name = "best.txt"

In [280]:
import lightgbm as lgb

def setup_prediction():
    """Load and setup model for prediction"""
    if IS_KAGGLE:
        model_path = os.path.join(trained_models_path, model_name)
    else:
        model_path = os.path.join(PROJ_DIR, "training", "trained_models", model_name)
    
    # Load model directly as a Booster
    model = lgb.Booster(model_file=model_path)
    
    return model
    
# Load model globally
model = setup_prediction()

### Predict API

In [281]:
# TODO: FLoat value in predictions? Type 64?

# Global variables
lags_ : pl.DataFrame | None = None

def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    global lags_
    
    if lags is not None:
        lags_ = lags

        # Update progress bar for new day if running locally
        if not IS_KAGGLE:
            progress_bar.update(1)

    # Preprocess and select needed columns
    processed = preprocess_features(test)

    # TODO: DEBUG IS HERE
    # Add debug print - only for last time_id of date_id 1
    # if (test.get_column('date_id')[0] == 1 and 
    #     test.get_column('time_id')[0] == 967):
    #     print("The day is")
    #     print(test.get_column('time_id')[0])
    #     print("\nSubmission data - Last moment of first API date (real date 1680):")
    #     # Group by symbol_id (first column of processed)
    #     for symbol in range(39):
    #         row_idx = np.where(processed[:, 0] == symbol)[0][0]
    #         features = [
    #             f"({symbol}, 'feature_{i:02d}'): {processed[row_idx, i+2]:.6f} (null: {processed[row_idx, i+2+79]})"
    #             for i in range(79)
    #         ]
    #         print(", ".join(features))

    if (test.get_column('date_id')[0] == 19 and 
        test.get_column('time_id')[0] == 967):
        predictions = model.predict(processed)
        print("\nAPI Submission - First 10 predictions for time 0 API day:")
        print("Symbol ID | Prediction  | Weight")
        print("-" * 40)
        for i in range(39):
            symbol_id = test.get_column('symbol_id')[i]
            print(f"   {symbol_id:2d}    | {predictions[i]:10.6f} | {test.get_column('weight')[i]:8.6f}")
    
    # Convert to numpy and predict
    predictions = model.predict(processed)
    
    # Create return DataFrame efficiently
    predictions = test.select('row_id').with_columns(
        pl.Series('responder_6', predictions, dtype=pl.Float64)
    )
    
    assert isinstance(predictions, pl.DataFrame | pd.DataFrame) # The predict function must return a DataFrame
    assert predictions.columns == ['row_id', 'responder_6']     # with columns 'row_id', 'responer_6'
    assert len(predictions) == len(test)                        # and as many rows as the test data.

    return predictions

### Submit

In [282]:
# Set up inference server
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

# Run based on environment
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:    
    inference_server.run_local_gateway(
        (
            os.path.join(data_path, "test.parquet"),
            os.path.join(data_path, "lags.parquet"),
        )
    )

    # Close progress bar if it exists
    if progress_bar is not None:
        progress_bar.close()
    
    # Local scoring
    print("Scoring submission...")
    predictions = pl.read_parquet("submission.parquet")                      # Read submission predictions
    test_data = pl.read_parquet(os.path.join(data_path, "test.parquet"))     # Read test data with actual values

    # Score only rows marked for scoring
    mask = test_data['is_scored']
    score = r2_score_weighted(
        test_data.filter(mask)['responder_6'].to_numpy(),
        predictions.filter(mask)['responder_6'].to_numpy(),
        test_data.filter(mask)['weight'].to_numpy()
    )
    print(f"Overall R² score: {score}")

Processing days:  95%|█████████▌| 20/21 [03:28<00:10, 10.95s/it]


API Submission - First 10 predictions for time 0 API day:
Symbol ID | Prediction  | Weight
----------------------------------------
    0    |  -0.006327 | 3.743546
    1    |  -0.007440 | 2.826357
    2    |  -0.007035 | 3.120292
    3    |  -0.006327 | 2.638314
    4    |  -0.006327 | 1.852919
    5    |  -0.006327 | 3.261852
    6    |  -0.006327 | 1.107478
    7    |  -0.006327 | 3.652727
    8    |  -0.006327 | 2.436262
    9    |  -0.007035 | 1.404590
   10    |  -0.007035 | 2.842925
   11    |  -0.006327 | 1.967007
   12    |  -0.006327 | 2.904539
   13    |  -0.006327 | 2.048854
   14    |  -0.006327 | 1.260267
   15    |  -0.008259 | 1.686074
   16    |  -0.008259 | 3.511533
   17    |  -0.006327 | 2.703924
   18    |  -0.008259 | 1.878528
   19    |  -0.006327 | 4.964283
   20    |  -0.007440 | 0.928589
   21    |  -0.008259 | 1.238956
   22    |  -0.006327 | 1.038181
   23    |  -0.007440 | 1.514470
   24    |  -0.007035 | 1.460937
   25    |  -0.006327 | 2.584827
   26    

Processing days:  95%|█████████▌| 20/21 [03:39<00:10, 10.96s/it]

Scoring submission...
Overall R² score: -0.0002239544922495451


In [283]:
# Overall R² score: -0.0026155209774680888 run with wrong blah

# Overall R² score: -0.0022625083960658987 with the correct blah

# Overall R² score: -0.0022625083960658987 zero fill off

# How are the fills on the blah compared to this fill, they must be wrong..